In [1]:
### 000 CONFIGURACION GENERAL
# -------------------------------
# CONFIGURACIÓN GENERAL
# -------------------------------
TIPO_ESTRUCTURA, Zona_climatica, exposicion, clase = "Suspensión Recta", "D", "C", "B" 
TITULO = f"Sdt_mas2_poov12_58mm2 Rev B"

# Guardado
tipoestructura_nombre_archivo = f"{TIPO_ESTRUCTURA.replace(' ', '_').replace('ó','o').replace('/','_').lower()}"
folder = f"{TITULO}/{tipoestructura_nombre_archivo}"

# Tipos: "Suspensión Recta", "Suspensión angular", "Retención / Ret. Angular", "Terminal"

# Usamos la LibCables para obtener los objetos cable
cable_conductor_id, cable_guardia_id = "Al/Ac 70/12", "OPGW FiberHome 24FO 58mm2"


# Parámetros del viento
Vmax = 38.9 # m/s
Vmed = 0.4 * Vmax
Vtormenta = 20.0
t_hielo = 0.010 # m
Q = 0.0613  
Zco, Zcg, Zca, Zes = 13.0, 13.0, 13.0, 10.0
Cf_cable, Cf_guardia, Cf_cadena, Cf_estructura = 1.0, 1.0, 0.9, 0.9

# Parámetros de diseño de línea
L_vano = 150.0 # m
alpha = 0.0  # grados, angulo max de quiebre
theta = 45.0 # grados, angulo de viento oblicuo, respecto transversal
A_cadena = 0.03  # m²
PCADENA = 10.5 # daN
PESTRUCTURA = 3900 # daN
A_estr_trans = 2 * 15 * (0.4 + 0.4 + 14.8 * 0.015) / 2 #+ 2 * 0.3 # m² # sumo 2 vinculos
A_estr_long = A_estr_trans + 3 * 0.5  # m², sumo 3 crucetas 

FORZAR_N_POSTES = 1  # 0=auto, 1=monoposte, 2=biposte, 3=triposte
FORZAR_ORIENTACION = "No" # Opciones: "Longitudinal", "Transversal", "No"
PRIORIDAD_DIMENSIONADO = "longitud_total" #(altura_libre: aumenta la long total manteniendo la altura libre fija, longitud_total: mantiene la longitud total fija y varia la altura libre)
# ================= CONFIGURACIÓN DISEÑO DE CABEZAL =================
TENSION, Zona_estructura = 33, "Rural"
DISPOSICION, TERNA, CANT_HG = "vertical", "Doble", 1
ALTURA_MINIMA_CABLE, LONGITUD_MENSULA_MINIMA_CONDUCTOR, LONGITUD_MENSULA_MINIMA_GUARDIA = 6.5, 1.3, 0.0
HADD = 2.4 ## Hadd = 0.4 m a la S base, 0.4 a la RA35dt, 0.9 a la Tdt+0.5
Lk = 0.75 # 0.75 # altura adicional, long. cadena oscilante
HADD_ENTRE_AMARRES = 0.2

# Configuración para mostrar/ocultar hipótesis C2 en graficos de reacciones
MOSTRAR_C2 = False
ANCHO_CRUCETA = 0.2 # m

# ================= CONFIGURACIÓN DE FLECHADO =================
SALTO_PORCENTUAL, PASO_AFINADO = 0.05, 0.005
OBJ_CONDUCTOR, OBJ_GUARDIA = 'FlechaMin', 'TiroMin'
RELFLECHA_MAX_GUARDIA = 0.95
RELFLECHA_SIN_VIENTO = True  # 

restricciones = {
    "conductor": {"tension_max_porcentaje": {"I": 0.25, "II": 0.40, "III": 0.40, "IV": 0.40, "V": 0.25}},
    "guardia": {"tension_max_porcentaje": {"I": 0.7, "II": 0.70, "III": 0.70, "IV": 0.7, "V": 0.7}, "relflecha_max": RELFLECHA_MAX_GUARDIA}
}

estados_climaticos = {
    "I": {"temperatura": 35, "descripcion": "Tmáx", "viento_velocidad": 0, "espesor_hielo": 0},
    "II": {"temperatura": -20, "descripcion": "Tmín", "viento_velocidad": 0, "espesor_hielo": 0},
    "III": {"temperatura": 10, "descripcion": "Vmáx", "viento_velocidad": Vmax, "espesor_hielo": 0},
    "IV": {"temperatura": -5, "descripcion": "Vmed", "viento_velocidad": Vmed, "espesor_hielo": t_hielo},
    "V": {"temperatura": 8, "descripcion": "TMA", "viento_velocidad": 0, "espesor_hielo": 0}
}

# ================= CONFIGURACIÓN GRÁFICOS =================
ZOOM_CABEZAL = 0.95
REEMPLAZAR_TITULO_GRAFICO = False
TITULO_REEMPLAZO = TITULO if REEMPLAZAR_TITULO_GRAFICO else TIPO_ESTRUCTURA

## Automaticas
Vn = TENSION


"""
NOTAS 
#####
- Se deben separar TOP e HG para permitir el HG 0.125 m mas bajo en retenciones y 0.135 m mas alto en suspensión, y defasaje longitudinal de 0.45
"""

'\nNOTAS \n#####\n- Se deben separar TOP e HG para permitir el HG 0.125 m mas bajo en retenciones y 0.135 m mas alto en suspensión, y defasaje longitudinal de 0.45\n'

In [2]:
### 001 IMPORTS PROPIOS
from CalculoCables import Cable_AEA, Elemento_AEA, LibCables    # Funciones de cálculo de cables
from CalculoEstructura import Estructura_AEA, NodoEstructural   # Funciones de cálculo de estructura
from HipotesisMaestro import hipotesis_maestro                  # Hipotesis de carga para CalculoEstructura
from DatosCables import datos_cables                            # Datos de cables para LibCables
from PostesHormigon import PostesHormigon                       # Librería de postes de hormigón (longitud / peso / resistencia / diametro en cima)
from Estructura3D import Estructura3D                           # Clase para graficar

In [3]:
### 002 CREAR OBJETOS CABLE, CADENA, ESTRUCTURA
# Datos de cables (de la celda 002 original)
DATOS_CABLES = datos_cables

# Crear lib de cables
lib_cables = LibCables()

# Crear objetos cable a partir de los datos
for nombre_cable, propiedades in DATOS_CABLES.items():
    cable = Cable_AEA(
        id_cable=nombre_cable,
        nombre=nombre_cable,
        propiedades=propiedades
    )
    lib_cables.agregar_cable(cable)

print("✅ OBJETOS CABLE CREADOS EXITOSAMENTE")
print("=" * 50)

# Mostrar cables disponibles
lib_cables.listar_cables()


cable_conductor = lib_cables.obtener_cable(cable_conductor_id)
cable_guardia = lib_cables.obtener_cable(cable_guardia_id)


print("✅ CONFIGURACIÓN GENERAL CARGADA")
print(f"Estructura: {TIPO_ESTRUCTURA}")
print(f"Conductor: {cable_conductor.nombre}")
print(f"Guardia: {cable_guardia.nombre}")
print(f"Vano: {L_vano} m")
print(f"Vmax: {Vmax} m/s, Vmed: {Vmed:.1f} m/s")

# Crear objetos cadena y estructura basados en la configuración
cadena = Elemento_AEA(
    id_elemento="cadena_aisladores",
    nombre="Cadena de Aisladores",
    area_transversal_m2=A_cadena,
    area_longitudinal_m2=A_cadena,  # Misma área en ambos sentidos
    Cf=Cf_cadena,
    Z=Zca,
    peso_daN=PCADENA
)

estructura = Elemento_AEA(
    id_elemento="estructura",
    nombre="Estructura",
    area_transversal_m2=A_estr_trans,
    area_longitudinal_m2=A_estr_long,  # Diferentes áreas
    Cf=Cf_estructura,
    Z=Zes,
    peso_daN=PESTRUCTURA
)

print("✅ OBJETOS Elemento_AEA CREADOS EXITOSAMENTE")
print("=" * 60)
print(f"Cadena: {cadena}")
print(f"Estructura: {estructura}")


# Variables globales para compatibilidad con celdas existentes
cable_conductor_obj = lib_cables.obtener_cable("Al/Ac 70/12")
cable_guardia_obj = lib_cables.obtener_cable("OPGW 07A378s 12FO 62mm2")

print(f"\n🔗 VARIABLES DE COMPATIBILIDAD:")
print(f"   - cable_conductor_obj: {cable_conductor_obj.nombre}")
print(f"   - cable_guardia_obj: {cable_guardia_obj.nombre}")

✅ OBJETOS CABLE CREADOS EXITOSAMENTE
📋 CABLES DISPONIBLES:
  - Ac 70: Ac, Ø11.0mm, Sección: 72.23mm², Peso: 0.6 daN/m
  - ADSS 1: Sintético, Ø15.0mm, Sección: 5.3mm², Peso: 0.185 daN/m
  - AlAc 435/55: Al/Ac, Ø28.8mm, Sección: 490.6mm², Peso: 1.653 daN/m
  - AlAc 300/50: Al/Ac, Ø24.5mm, Sección: 353.7mm², Peso: 1.20663 daN/m
  - Ac 50: Ac, Ø9.0mm, Sección: 49.49mm², Peso: 0.3924 daN/m
  - Al/Ac 70/12: Al/Ac, Ø11.7mm, Sección: 81.3mm², Peso: 0.28746 daN/m
  - OPGW 07A378s 12FO 62mm2: ACS, Ø10.7mm, Sección: 62.4mm², Peso: 0.359 daN/m
  - OPGW 44F70s 24FO 120mm2: ACS/A, Ø14.4mm, Sección: 120.0mm², Peso: 0.68642 daN/m
  - OPGW FiberHome 24FO 58mm2: ACS/A, Ø10.5mm, Sección: 57.73mm², Peso: 0.289277 daN/m
✅ CONFIGURACIÓN GENERAL CARGADA
Estructura: Suspensión Recta
Conductor: Al/Ac 70/12
Guardia: OPGW FiberHome 24FO 58mm2
Vano: 150.0 m
Vmax: 38.9 m/s, Vmed: 15.6 m/s
✅ OBJETOS Elemento_AEA CREADOS EXITOSAMENTE
Cadena: Elemento cadena_aisladores: Cadena de Aisladores - A_trans: 0.030m², A_long

In [4]:
### 003 CMC Y CARGAS
import pandas as pd
import math
import os

print("🔧 INICIANDO CÁLCULO COMPLETO OPTIMIZADO")
print("=" * 60)
print(f"Estructura: {TIPO_ESTRUCTURA}")
print(f"Conductor: {cable_conductor.nombre}")
print(f"Guardia: {cable_guardia.nombre}")
print(f"Vano: {L_vano} m")
print(f"Vmax: {Vmax} m/s, Vmed: {Vmed:.1f} m/s")

# ================= CÁLCULO OPTIMIZADO DE CARGAS DE VIENTO =================
print("\n🌬️ CALCULANDO CARGAS DE VIENTO OPTIMIZADO...")

# Cache de resultados para evitar cálculos duplicados
cache_viento = {}

def calcular_carga_viento_cable(cable_obj, V, Z, Cf, d_eff, phi_rel, factor_vano):
    """Función optimizada para cálculo de viento en cables"""
    clave_cache = f"{cable_obj.id}_{V}_{phi_rel}"
    
    if clave_cache not in cache_viento:
        resultado = cable_obj.cargaViento(
            V=V, phi_rel_deg=phi_rel, exp=exposicion, clase=clase,
            Zc=Z, Cf=Cf, L_vano=factor_vano, d_eff=d_eff
        )
        cache_viento[clave_cache] = {
            'unitario': resultado["fuerza_daN_per_m"],
            'total': resultado["fuerza_daN_per_m"] * factor_vano
        }
    
    return cache_viento[clave_cache]

# Diámetros efectivos
d_conductor = cable_conductor.diametro_m
d_guardia = cable_guardia.diametro_m
d_conductor_eff = cable_conductor.diametro_equivalente(t_hielo)
d_guardia_eff = cable_guardia.diametro_equivalente(t_hielo)

alpha_mitad = alpha / 2.0
rows = []

def calcular_angulo_relativo_cable(theta_grados, alpha_mitad_grados, lado, direccion_viento):
    """Calcula el ángulo relativo entre el viento y el cable - OPTIMIZADO"""
    if direccion_viento == "transversal":
        return 90.0 - alpha_mitad_grados
    elif direccion_viento == "longitudinal":
        return alpha_mitad_grados
    else:  # oblicuo
        if lado == 1:
            return 90.0 - theta_grados + alpha_mitad_grados
        else:  # lado 2
            return 90.0 - theta_grados - alpha_mitad_grados

# Generar escenarios de viento de forma más eficiente
escenarios_viento = [
    # (vel_label, V, direccion, factor_vano, es_oblicuo)
    ("Vmax - Transversal", Vmax, "transversal", L_vano, False),
    ("Vmax - Longitudinal", Vmax, "longitudinal", L_vano, False),
    ("Vmax - Oblicua", Vmax, "oblicuo", L_vano/2, True),
    ("Vmed - Transversal", Vmed, "transversal", L_vano, False),
    ("Vmed - Longitudinal", Vmed, "longitudinal", L_vano, False),
    ("Vmed - Oblicua", Vmed, "oblicuo", L_vano/2, True)
]

cables_config = [
    ("Conductor", cable_conductor, Zco, Cf_cable, d_conductor, d_conductor_eff),
    ("Cable Guardia", cable_guardia, Zcg, Cf_guardia, d_guardia, d_guardia_eff)
]

for vel_label, V, direccion, factor_vano, es_oblicuo in escenarios_viento:
    for elemento, cable_obj, Z, Cf, d_base, d_eff in cables_config:
        d_use = d_eff if "Vmed" in vel_label else d_base
        
        if es_oblicuo:
            for lado in [1, 2]:
                # Calcular ángulo relativo
                phi_rel = calcular_angulo_relativo_cable(theta, alpha_mitad, lado, direccion)
                
                # Fuerza de viento sobre el cable (medio vano)
                resultado = calcular_carga_viento_cable(cable_obj, V, Z, Cf, d_use, phi_rel, factor_vano)
                fuerza_cable = resultado['total']
                
                # Descomposición en componentes estructurales usando ángulo de línea (α/2)
                alpha_mitad_rad = math.radians(alpha_mitad)
                
                # Componente transversal = F_cable × sin(α/2)
                f_estructura_trans = fuerza_cable * math.cos(alpha_mitad_rad)
                
                # Componente longitudinal = F_cable × cos(α/2)  
                f_estructura_long = fuerza_cable * math.sin(alpha_mitad_rad)
                
                # Fuerza sobre el cable
                rows.append({
                    "Velocidad_label": vel_label, "Velocidad_m_s": V, "Dirección": "Oblicuo",
                    "Elemento": elemento, "Descripción": f"Sobre CABLE (Lado {lado})",
                    "Fu_daN_per_m": round(resultado['unitario'], 2),
                    "F_total_daN": round(fuerza_cable, 2),
                    "Angulo": f"{phi_rel:.1f}°"
                })
                
                # Efecto sobre estructura - componente transversal
                rows.append({
                    "Velocidad_label": vel_label, "Velocidad_m_s": V, "Dirección": "Transversal",
                    "Elemento": elemento, "Descripción": f"Sobre ESTRUCTURA (Lado {lado})",
                    "Fu_daN_per_m": None,
                    "F_total_daN": round(f_estructura_trans, 2),
                    "Angulo": "Estruct."
                })
                
                # Efecto sobre estructura - componente longitudinal
                rows.append({
                    "Velocidad_label": vel_label, "Velocidad_m_s": V, "Dirección": "Longitudinal",
                    "Elemento": elemento, "Descripción": f"Sobre ESTRUCTURA (Lado {lado})",
                    "Fu_daN_per_m": None,
                    "F_total_daN": round(f_estructura_long, 2),
                    "Angulo": "Estruct."
                })
        else:
            phi_rel = calcular_angulo_relativo_cable(theta, alpha_mitad, 1, direccion)
            resultado = calcular_carga_viento_cable(cable_obj, V, Z, Cf, d_use, phi_rel, factor_vano)
            
            direccion_label = "Transversal" if direccion == "transversal" else "Longitudinal"
            rows.append({
                "Velocidad_label": vel_label, "Velocidad_m_s": V, "Dirección": direccion_label,
                "Elemento": elemento, "Descripción": f"Sobre CABLE ({direccion_label})",
                "Fu_daN_per_m": round(resultado['unitario'], 2), "F_total_daN": round(resultado['total'], 2), "Angulo": f"{phi_rel:.1f}°"
            })

# Elementos estructurales (cadenas y estructura)
elementos_config = [
    (cadena, "Cadena de Aisladores"),
    (estructura, "Estructura")
]

for vel_label, V in [("Vmax - Transversal", Vmax), ("Vmax - Longitudinal", Vmax),
                     ("Vmed - Transversal", Vmed), ("Vmed - Longitudinal", Vmed)]:
    direccion = "Transversal" if "Transversal" in vel_label else "Longitudinal"
    angulo_theta = 90 if direccion == "Transversal" else 0
    
    for elemento_obj, nombre in elementos_config:
        resultado = elemento_obj.cargaViento(
            V=V, theta_deg=angulo_theta, exp=exposicion, clase=clase, Q=Q, L_vano=L_vano
        )
        
        rows.append({
            "Velocidad_label": vel_label, "Velocidad_m_s": V, "Dirección": direccion,
            "Elemento": nombre, "Descripción": f"Sobre {nombre.split()[0]} ({direccion})",
            "Fu_daN_per_m": None, "F_total_daN": round(resultado["fuerza_total_daN"], 2), "Angulo": f"{angulo_theta:.1f}°"
        })

# Viento oblicuo en estructura
for vel_label, V in [("Vmax - Oblicua", Vmax), ("Vmed - Oblicua", Vmed)]:
    resultado_estructura = estructura.cargaViento(V=V, theta_deg=theta, exp=exposicion, clase=clase, Q=Q, L_vano=L_vano)
    
    # Ahora usamos las componentes directas del resultado
    f_total_trans = resultado_estructura["fuerza_transversal_daN"]
    f_total_long = resultado_estructura["fuerza_longitudinal_daN"]
    
    rows.extend([
        {
            "Velocidad_label": vel_label, "Velocidad_m_s": V, "Dirección": "Transversal",
            "Elemento": "Estructura", "Descripción": "Sobre Estructura (Oblicuo)",
            "Fu_daN_per_m": None, "F_total_daN": round(f_total_trans, 2), "Angulo": f"{theta}°"
        },
        {
            "Velocidad_label": vel_label, "Velocidad_m_s": V, "Dirección": "Longitudinal", 
            "Elemento": "Estructura", "Descripción": "Sobre Estructura (Oblicuo)",
            "Fu_daN_per_m": None, "F_total_daN": round(f_total_long, 2), "Angulo": f"{theta}°"
        }
    ])

# Crear DataFrame con ambas columnas
df_cargas = pd.DataFrame(rows)

print("📊 CARGA DE VIENTO - RESUMEN COMPLETO")
print("=" * 100)
display(df_cargas)

# Guardar cargas
os.makedirs(folder, exist_ok=True)
df_cargas.to_csv(f"{folder}/1_{tipoestructura_nombre_archivo}_Cargas_Viento.csv", index=False, encoding='utf-8')
print(f"💾 Cargas guardadas en: {folder}/1_{tipoestructura_nombre_archivo}_Cargas_Viento.csv")

# ================= CÁLCULO MECÁNICO DE CABLES (MANTENIDO) =================
print("\n🔧 CALCULANDO CÁLCULO MECÁNICO DE CABLES...")

parametros_viento = {"exposicion": exposicion, "clase": clase, "Zc": Zco, "Cf": Cf_cable, "L_vano": L_vano}
parametros_viento_guardia = {**parametros_viento, "Zc": Zcg, "Cf": Cf_guardia}

print("🔧 CALCULANDO CONDUCTOR...")
df_conductor, resultados_conductor, estado_limitante_cond = cable_conductor.calculo_mecanico(
    vano=L_vano, estados_climaticos=estados_climaticos, parametros_viento=parametros_viento,
    restricciones=restricciones["conductor"], objetivo=OBJ_CONDUCTOR, es_guardia=False,
    flecha_max_permitida=3.0, salto_porcentual=SALTO_PORCENTUAL, paso_afinado=PASO_AFINADO, relflecha_sin_viento=RELFLECHA_SIN_VIENTO
)

print("✅ CONDUCTOR COMPLETADO")
# Formatear floats en el DataFrame del conductor
for col in df_conductor.columns:
    if df_conductor[col].dtype == 'float64':
        df_conductor[col] = df_conductor[col].round(2)
display(df_conductor)

# Calcular guardia
flecha_max_conductor = max([r["flecha_vertical_m"] for r in resultados_conductor.values()])
flecha_max_guardia = flecha_max_conductor * RELFLECHA_MAX_GUARDIA

print("🔧 CALCULANDO GUARDIA...")
df_guardia, resultados_guardia, estado_limitante_guard = cable_guardia.calculo_mecanico(
    vano=L_vano, estados_climaticos=estados_climaticos, parametros_viento=parametros_viento_guardia,
    restricciones=restricciones["guardia"], objetivo=OBJ_GUARDIA, es_guardia=True,
    flecha_max_permitida=flecha_max_guardia, resultados_conductor=resultados_conductor,
    salto_porcentual=SALTO_PORCENTUAL, paso_afinado=PASO_AFINADO, relflecha_sin_viento=RELFLECHA_SIN_VIENTO
)

print("✅ GUARDIA COMPLETADO")
# Formatear floats en el DataFrame del guardia
for col in df_guardia.columns:
    if df_guardia[col].dtype == 'float64':
        df_guardia[col] = df_guardia[col].round(2)
display(df_guardia)

# Combinar y guardar resultados CMC
df_combinado = pd.concat([df_conductor, df_guardia], ignore_index=True)
df_combinado.to_csv(f"{folder}/2_{tipoestructura_nombre_archivo}_CMC.csv", index=False, encoding='utf-8')
print(f"💾 CMC guardado en: {folder}/2_{tipoestructura_nombre_archivo}_CMC.csv")

# ================= LISTADO OPTIMIZADO DE CARGAS =================
print("\n📋 GENERANDO LISTADO OPTIMIZADO DE CARGAS...")

# Cache para cargas ya calculadas
cargas_cache = {}

def obtener_carga_optimizada(elemento, direccion, velocidad_label, desc_adicional=""):
    """Función optimizada para obtener cargas del DataFrame"""
    clave_cache = f"{elemento}_{direccion}_{velocidad_label}_{desc_adicional}"
    
    if clave_cache not in cargas_cache:
        try:
            filtro = (df_cargas['Elemento'] == elemento) & (df_cargas['Dirección'] == direccion) & (df_cargas['Velocidad_label'] == velocidad_label)
            if desc_adicional:
                filtro = filtro & (df_cargas['Descripción'].str.contains(desc_adicional))
            
            # Usar F_total_daN para todos los elementos
            resultado = df_cargas[filtro]['F_total_daN'].iloc[0] if not df_cargas[filtro].empty else 0.0
            cargas_cache[clave_cache] = round(resultado, 2) if resultado is not None else 0.0
        except:
            cargas_cache[clave_cache] = 0.0
    
    return cargas_cache[clave_cache]

# Datos base optimizados
peso_conductor_base = cable_conductor.peso_unitario_dan_m
peso_guardia_base = cable_guardia.peso_unitario_dan_m
peso_hielo_conductor = cable_conductor._calcular_peso_hielo(t_hielo, 900)
peso_hielo_guardia = cable_guardia._calcular_peso_hielo(t_hielo, 900)

Pc = round(peso_conductor_base * L_vano, 2)
Pcg = round(peso_guardia_base * L_vano, 2)
Pch = round(peso_hielo_conductor * L_vano, 2)
Pcgh = round(peso_hielo_guardia * L_vano, 2)
CS = 220.00

estados_climaticos_map = {"I": "Tmax", "II": "Tmin", "III": "Vmax", "IV": "Vmed", "V": "Tma"}

cargas_data = []
id_counter = 1

# Función auxiliar para agregar cargas
def agregar_carga(elemento, codigo, descripcion, estado_climatico, magnitud, direccion):
    global id_counter
    # Formatear magnitud a 2 decimales si es float
    if isinstance(magnitud, float):
        magnitud = round(magnitud, 2)
    
    cargas_data.append({
        "ID": id_counter, "Elemento": elemento, "Código": codigo, 
        "Carga": descripcion, "Estado Climático": estado_climatico, 
        "Magnitud": magnitud, "Unidad": "daN", "Direccion": direccion
    })
    id_counter += 1

# Cargas verticales
cargas_verticales = [
    ("Conductor", "Pc", "Peso de Gravivano", "NA", Pc, "Vertical"),
    ("Cable Guardia", "Pcg", "Peso de Gravivano", "NA", Pcg, "Vertical"),
    ("Conductor", "Pch", "Peso adicional por hielo", "NA", Pch, "Vertical"),
    ("Cable Guardia", "Pcgh", "Peso Adicional por hielo", "NA", Pcgh, "Vertical"),
    ("Cadena", "Pa", "Peso de Cadena de Aisladores y Herrajes", "NA", PCADENA, "Vertical"),
    ("Estructura", "CS", "Carga de Servicio", "NA", CS, "Vertical")
]

for carga in cargas_verticales:
    agregar_carga(*carga)

# Viento máximo - cables
viento_max_configs = [
    ("Conductor", "Transversal", "Vc", "Viento Máximo en Eolovano", "Vmax"),
    ("Conductor", "Longitudinal", "VcL", "Viento Máximo en Eolovano", "Vmax"),
    ("Cable Guardia", "Transversal", "Vcg", "Viento Máximo en Eolovano", "Vmax"), 
    ("Cable Guardia", "Longitudinal", "VcgL", "Viento Máximo en Eolovano", "Vmax")
]

for elemento, direccion, codigo, descripcion, estado in viento_max_configs:
    magnitud = obtener_carga_optimizada(elemento, direccion, f"{estado} - {direccion}")
    agregar_carga(elemento, codigo, descripcion, estado, magnitud, direccion)

# Viento máximo oblicuo - cables
for elemento in ["Conductor", "Cable Guardia"]:
    for lado in [1, 2]:
        codigo_base = "Vc_o" if elemento == "Conductor" else "Vcg_o"
        desc_base = f"Viento Máximo en Eolovano - Oblícuo - Lado {lado}"
        
        # Componente transversal
        magnitud_trans = obtener_carga_optimizada(elemento, "Transversal", "Vmax - Oblicua", f"Lado {lado}")
        agregar_carga(elemento, f"{codigo_base}_t_{lado}", desc_base, "Vmax", magnitud_trans, "Transversal")
        
        # Componente longitudinal  
        magnitud_long = obtener_carga_optimizada(elemento, "Longitudinal", "Vmax - Oblicua", f"Lado {lado}")
        agregar_carga(elemento, f"{codigo_base}_l_{lado}", desc_base, "Vmax", magnitud_long, "Longitudinal")

# Viento medio - cables
viento_medio_configs = [
    ("Conductor", "Transversal", "Vcmed", "Viento Medio en Eolovano", "Vmed"),
    ("Conductor", "Longitudinal", "VcmedL", "Viento Medio en Eolovano", "Vmed"),
    ("Cable Guardia", "Transversal", "Vcgmed", "Viento Medio en Eolovano", "Vmed"),
    ("Cable Guardia", "Longitudinal", "VcgmedL", "Viento Medio en Eolovano", "Vmed")
]

for elemento, direccion, codigo, descripcion, estado in viento_medio_configs:
    magnitud = obtener_carga_optimizada(elemento, direccion, f"{estado} - {direccion}")
    agregar_carga(elemento, codigo, descripcion, estado, magnitud, direccion)

# Viento medio oblicuo - cables
for elemento in ["Conductor", "Cable Guardia"]:
    for lado in [1, 2]:
        codigo_base = "Vcmed_o" if elemento == "Conductor" else "Vcgmed_o"
        desc_base = f"Viento Medio en Eolovano - Oblícuo - Lado {lado}"
        
        magnitud_trans = obtener_carga_optimizada(elemento, "Transversal", "Vmed - Oblicua", f"Lado {lado}")
        agregar_carga(elemento, f"{codigo_base}_t_{lado}", desc_base, "Vmed", magnitud_trans, "Transversal")
        
        magnitud_long = obtener_carga_optimizada(elemento, "Longitudinal", "Vmed - Oblicua", f"Lado {lado}")
        agregar_carga(elemento, f"{codigo_base}_l_{lado}", desc_base, "Vmed", magnitud_long, "Longitudinal")

# Elementos estructurales - viento
elementos_estructurales = [
    ("Cadena", "VaT", "VaL", "VamedT", "VamedL", "Cadena de Aisladores"),
    ("Estructura", "VeT", "VeL", "Vemedt", "Vemedl", "Estructura")
]

for elemento, codigo_t_max, codigo_l_max, codigo_t_med, codigo_l_med, nombre_completo in elementos_estructurales:
    # Viento máximo
    magnitud_t_max = obtener_carga_optimizada(nombre_completo, "Transversal", "Vmax - Transversal")
    magnitud_l_max = obtener_carga_optimizada(nombre_completo, "Longitudinal", "Vmax - Longitudinal")
    
    agregar_carga(elemento, codigo_t_max, f"Viento Máximo en {nombre_completo}", "Vmax", magnitud_t_max, "Transversal")
    agregar_carga(elemento, codigo_l_max, f"Viento Máximo en {nombre_completo}", "Vmax", magnitud_l_max, "Longitudinal")
    
    # Viento medio
    magnitud_t_med = obtener_carga_optimizada(nombre_completo, "Transversal", "Vmed - Transversal")
    magnitud_l_med = obtener_carga_optimizada(nombre_completo, "Longitudinal", "Vmed - Longitudinal")
    
    agregar_carga(elemento, codigo_t_med, f"Viento Medio en {nombre_completo}", "Vmed", magnitud_t_med, "Transversal")
    agregar_carga(elemento, codigo_l_med, f"Viento Medio en {nombre_completo}", "Vmed", magnitud_l_med, "Longitudinal")

# Viento oblicuo en estructura
for vel_label, estado in [("Vmax - Oblicua", "Vmax"), ("Vmed - Oblicua", "Vmed")]:
    codigo_transv = "VeoT" if estado == "Vmax" else "VemedoT"
    codigo_long = "VeoL" if estado == "Vmax" else "VemedoL"
    
    magnitud_trans = obtener_carga_optimizada("Estructura", "Transversal", vel_label, "Oblicuo")
    magnitud_long = obtener_carga_optimizada("Estructura", "Longitudinal", vel_label, "Oblicuo")
    
    agregar_carga("Estructura", codigo_transv, f"Viento {estado} sobre Estructura - Oblicuo - Comp. Transv.", estado, magnitud_trans, "Transversal")
    agregar_carga("Estructura", codigo_long, f"Viento {estado} sobre Estructura - Oblicuo - Comp. Long.", estado, magnitud_long, "Longitudinal")

# Tiros de cables
def obtener_tiro_estado(resultados_dict, estado, componente="longitudinal"):
    if estado in resultados_dict:
        tiro_total = resultados_dict[estado]["tiro_daN"]
        alpha_rad = math.radians(alpha / 2.0)
        if componente == "transversal":
            return round(tiro_total * math.sin(alpha_rad), 2)
        else:
            return round(tiro_total * math.cos(alpha_rad), 2)
    return 0.0

for elemento, resultados_dict, prefijo in [
    ("Conductor", resultados_conductor, "c"),
    ("Cable Guardia", resultados_guardia, "cg")
]:
    for estado, nombre_estado in estados_climaticos_map.items():
        for componente, sufijo in [("transversal", "t"), ("longitudinal", "l")]:
            codigo = f"T{nombre_estado.lower()}_{prefijo}_{sufijo}"
            magnitud = obtener_tiro_estado(resultados_dict, estado, componente)
            descripcion = f"Componente {componente.title()} - Tiro {nombre_estado.lower()} de {'conductor' if elemento == 'Conductor' else 'cable de guardia'}"
            
            agregar_carga(elemento, codigo, descripcion, nombre_estado, magnitud, componente.title())

# Crear y mostrar DataFrame final
df_cargas_totales = pd.DataFrame(cargas_data)

print(f"\n📊 RESUMEN DE CARGAS TOTALES - {TIPO_ESTRUCTURA}")
print("=" * 100)

# Configurar pandas para mostrar todas las filas y columnas sin truncar
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)  # Aumentar ancho máximo de columnas
pd.set_option('display.float_format', '{:,.2f}'.format)
display(df_cargas_totales)

# Guardar resultados
ruta_guardado = f"{folder}/3_{tipoestructura_nombre_archivo}_LISTA_TOTAL_DE_CARGAS.csv"
df_cargas_totales.to_csv(ruta_guardado, index=False, encoding='utf-8')

print(f"\n💾 Listado de cargas guardado en: {ruta_guardado}")
print(f"📊 Resumen de optimización:")
print(f"   - Cálculos de viento en cache: {len(cache_viento)}")
print(f"   - Cargas en cache: {len(cargas_cache)}")
print("🎯 CÁLCULO COMPLETO OPTIMIZADO FINALIZADO EXITOSAMENTE")

🔧 INICIANDO CÁLCULO COMPLETO OPTIMIZADO
Estructura: Suspensión Recta
Conductor: Al/Ac 70/12
Guardia: OPGW FiberHome 24FO 58mm2
Vano: 150.0 m
Vmax: 38.9 m/s, Vmed: 15.6 m/s

🌬️ CALCULANDO CARGAS DE VIENTO OPTIMIZADO...
📊 CARGA DE VIENTO - RESUMEN COMPLETO


,Velocidad_label,Velocidad_m_s,Dirección,Elemento,Descripción,Fu_daN_per_m,F_total_daN,Angulo
0,Vmax - Transversal,38.90,Transversal,Conductor,Sobre CABLE (Transversal),0.88,132.08,90.0°
1,Vmax - Transversal,38.90,Transversal,Cable Guardia,Sobre CABLE (Transversal),0.79,118.54,90.0°
2,Vmax - Longitudinal,38.90,Longitudinal,Conductor,Sobre CABLE (Longitudinal),0.00,0.00,0.0°
3,Vmax - Longitudinal,38.90,Longitudinal,Cable Guardia,Sobre CABLE (Longitudinal),0.00,0.00,0.0°
4,Vmax - Oblicua,38.90,Oblicuo,Conductor,Sobre CABLE (Lado 1),0.67,50.20,45.0°
5,Vmax - Oblicua,38.90,Transversal,Conductor,Sobre ESTRUCTURA (Lado 1),NaN,50.20,Estruct.
6,Vmax - Oblicua,38.90,Longitudinal,Conductor,Sobre ESTRUCTURA (Lado 1),NaN,0.00,Estruct.
7,Vmax - Oblicua,38.90,Oblicuo,Conductor,Sobre CABLE (Lado 2),0.67,50.20,45.0°
8,Vmax - Oblicua,38.90,Transversal,Conductor,Sobre ESTRUCTURA (Lado 2),NaN,50.20,Estruct.
9,Vmax - Oblicua,38.90,Longitudinal,Conductor,Sobre ESTRUCTURA (Lado 2),NaN,0.00,Estruct.


💾 Cargas guardadas en: Sdt_mas2_poov12_58mm2 Rev B/suspension_recta/1_suspension_recta_Cargas_Viento.csv

🔧 CALCULANDO CÁLCULO MECÁNICO DE CABLES...
🔧 CALCULANDO CONDUCTOR...

🔧 Cálculo mecánico para Al/Ac 70/12 (Conductor)
   Objetivo: FlechaMin, Vano: 150.0 m
   ⚠️  Parámetros ignorados (se usan valores fijos del algoritmo): ['salto_porcentual', 'paso_afinado']
  🎯 Objetivo: FlechaMin (minimizar flecha, aumentar tensión)
  📈 Tensión inicial: 0.323506 daN/mm² (1% de rotura)
  🔍 Iniciando búsqueda incremental: t_inicial=0.323506 daN/mm², paso=0.323506, sentido=↑, máximo 10000 iteraciones
    ✓ Iteración 0: t=0.323506 VÁLIDA
    ✗ Iteración 7: t=2.588044 VIOLADA por tension en estado IV
  🔄 Restricción violada en la búsqueda principal
     Última válida: t=2.264539, violación por tension en IV
  🎯 Iniciando ajuste fino triple CORREGIDO
     Rango ajuste: 2.264539 a 2.588044, sentido=↑
     Fase 1: Saltos del 1% (0.323506 daN/mm²)
       Fase 1: Violación en t=2.588044
       Fase 1 comp

,Cable,Descrip.,Temperatura [°C],Esp. manguito hielo [cm],Vano Regulador [m],Carga Peso [daN/m],Carga Hielo [daN/m],Carga Viento [daN/m],Carga vectorial [daN/m],Tensión [daN/mm2],Tiro [daN],Flecha [m],Flecha Vertical [m],% rotura,Estado determinante
0,Conductor,Tmáx,35,0.0,150.0,0.29,0.0,0.00,0.29,4.85,394.04,2.05,2.05,14.98,
1,Conductor,Tmín,-20,0.0,150.0,0.29,0.0,0.00,0.29,9.93,806.91,1.00,1.00,30.68,
2,Conductor,Vmáx,10,0.0,150.0,0.29,0.0,0.88,0.93,11.60,943.34,2.76,0.86,35.87,
3,Conductor,Vmed,-5,1.0,150.0,0.29,0.6,0.38,0.97,12.94,1051.83,2.59,2.38,39.99,🟡
4,Conductor,TMA,8,0.0,150.0,0.29,0.0,0.00,0.29,6.85,557.29,1.45,1.45,21.19,


🔧 CALCULANDO GUARDIA...

🔧 Cálculo mecánico para OPGW FiberHome 24FO 58mm2 (Guardia)
   Objetivo: TiroMin, Vano: 150.0 m
   ⚠️  Parámetros ignorados (se usan valores fijos del algoritmo): ['salto_porcentual', 'paso_afinado']
  🎯 Objetivo: TiroMin (minimizar tiro, disminuir tensión)
  📉 Tensión inicial: 42.802702 daN/mm² (70% de rotura)
  🔍 Iniciando búsqueda incremental: t_inicial=42.802702 daN/mm², paso=0.611467, sentido=↓, máximo 10000 iteraciones
    ✓ Iteración 0: t=42.802702 VÁLIDA
    ✗ Iteración 64: t=3.668803 VIOLADA por relflecha en estado II
  🔄 Restricción violada en la búsqueda principal
     Última válida: t=4.280270, violación por relflecha en II
  🎯 Iniciando ajuste fino triple CORREGIDO
     Rango ajuste: 3.668803 a 4.280270, sentido=↓
     Fase 1: Saltos del 1% (0.611467 daN/mm²)
       Fase 1 completada: última válida t=3.668803
       Inicio Fase 2 desde: t=4.280270
     Fase 2: Saltos del 0.1% (0.061147 daN/mm²)
       Fase 2: Violación en t=4.096830
       Fase 2 c

,Cable,Descrip.,Temperatura [°C],Esp. manguito hielo [cm],Vano Regulador [m],Carga Peso [daN/m],Carga Hielo [daN/m],Carga Viento [daN/m],Carga vectorial [daN/m],Tensión [daN/mm2],Tiro [daN],Flecha [m],Flecha Vertical [m],% rotura,Estado determinante,Rel. flecha
0,Guardia,Tmáx,35,0.0,150.0,0.29,0.00,0.00,0.29,8.09,467.07,1.74,1.74,13.23,,0.85
1,Guardia,Tmín,-20,0.0,150.0,0.29,0.00,0.00,0.29,14.66,846.16,0.96,0.96,23.97,🟡,0.96
2,Guardia,Vmáx,10,0.0,150.0,0.29,0.00,0.79,0.84,16.43,948.74,2.49,0.86,26.88,,1.00
3,Guardia,Vmed,-5,1.0,150.0,0.29,0.57,0.37,0.93,18.63,1075.26,2.44,2.24,30.46,,0.94
4,Guardia,TMA,8,0.0,150.0,0.29,0.00,0.00,0.29,10.89,628.92,1.29,1.29,17.82,,0.89


💾 CMC guardado en: Sdt_mas2_poov12_58mm2 Rev B/suspension_recta/2_suspension_recta_CMC.csv

📋 GENERANDO LISTADO OPTIMIZADO DE CARGAS...

📊 RESUMEN DE CARGAS TOTALES - Suspensión Recta


,ID,Elemento,Código,Carga,Estado Climático,Magnitud,Unidad,Direccion
0,1,Conductor,Pc,Peso de Gravivano,NA,43.12,daN,Vertical
1,2,Cable Guardia,Pcg,Peso de Gravivano,NA,43.39,daN,Vertical
2,3,Conductor,Pch,Peso adicional por hielo,NA,90.28,daN,Vertical
3,4,Cable Guardia,Pcgh,Peso Adicional por hielo,NA,85.29,daN,Vertical
4,5,Cadena,Pa,Peso de Cadena de Aisladores y Herrajes,NA,10.50,daN,Vertical
5,6,Estructura,CS,Carga de Servicio,NA,220.00,daN,Vertical
6,7,Conductor,Vc,Viento Máximo en Eolovano,Vmax,132.08,daN,Transversal
7,8,Conductor,VcL,Viento Máximo en Eolovano,Vmax,0.00,daN,Longitudinal
8,9,Cable Guardia,Vcg,Viento Máximo en Eolovano,Vmax,118.54,daN,Transversal
9,10,Cable Guardia,VcgL,Viento Máximo en Eolovano,Vmax,0.00,daN,Longitudinal



💾 Listado de cargas guardado en: Sdt_mas2_poov12_58mm2 Rev B/suspension_recta/3_suspension_recta_LISTA_TOTAL_DE_CARGAS.csv
📊 Resumen de optimización:
   - Cálculos de viento en cache: 12
   - Cargas en cache: 36
🎯 CÁLCULO COMPLETO OPTIMIZADO FINALIZADO EXITOSAMENTE


In [ ]:
### 004: CALCULO DE ESTRUCTURA
# ================= EJEMPLO DE USO =================
print("\n ESTRUCTURA_AEA COMPLETA")
print("=" * 80)

# FLECHAS MÁXIMAS A PARTIR DE LOS RESULTADOS DEL CÁLCULO MECÁNICO
fmax_conductor = max([r["flecha_vertical_m"] for r in resultados_conductor.values()])
fmax_guardia = max([r["flecha_vertical_m"] for r in resultados_guardia.values()])

# Crear estructura
estructura_poo = Estructura_AEA(
    tipo_estructura=TIPO_ESTRUCTURA,
    tension_nominal=TENSION,
    zona_estructura=Zona_estructura,
    disposicion=DISPOSICION,
    terna=TERNA,
    cant_hg=CANT_HG,
    alpha_quiebre=alpha,
    altura_minima_cable=ALTURA_MINIMA_CABLE,
    long_mensula_min_conductor=LONGITUD_MENSULA_MINIMA_CONDUCTOR,
    long_mensula_min_guardia=LONGITUD_MENSULA_MINIMA_GUARDIA,
    hadd=HADD,
    hadd_entre_amarres=HADD_ENTRE_AMARRES,
    lk=Lk,
    ancho_cruceta=ANCHO_CRUCETA,
    cable_conductor=cable_conductor,
    cable_guardia=cable_guardia,
    peso_estructura=PESTRUCTURA,
    peso_cadena=PCADENA
)

# Dimensionar (usando flechas de cálculos anteriores)
estructura_poo.dimensionar_unifilar(L_vano, fmax_conductor, fmax_guardia, df_cargas_totales)
parametros_cabezal = estructura_poo.obtener_parametros_cabezal()
print(parametros_cabezal.T)  # Transponer para mejor visualización

# Imprimir datos en consola
estructura_poo.imprimir_datos_dimensionamiento(L_vano, flecha_max_conductor, fmax_guardia)

# Graficar
estructura_poo.graficar_estructura(zoom_cabezal=ZOOM_CABEZAL, titulo_reemplazo=TITULO_REEMPLAZO)
estructura_poo.graficar_cabezal(zoom_cabezal=ZOOM_CABEZAL*1.5, titulo_reemplazo=TITULO_REEMPLAZO)

# Asignar cargas PASANDO hipotesis_maestro como argumento
estructura_poo.asignar_cargas_hipotesis(
    df_cargas_totales, 
    resultados_conductor, 
    resultados_guardia, 
    L_vano, 
    hipotesis_maestro,  # ← Viene del notebook
    t_hielo  # 
)

# Generar DataFrame de cargas
df_cargas_poo = estructura_poo.generar_dataframe_cargas()

# Guardar resultados
estructura_poo.guardar_resultados(folder)
nodes_key = estructura_poo.nodes_key
cargas_key = estructura_poo.cargas_key
altura_total = estructura_poo.dimensiones.get('altura_total', 0)

print(f"\n✅ ESTRUCTURA CALCULADA")


In [ ]:
### 005: REACCIONES Y TIRO EN CIMA

def main():
    
    # 1. CALCULAR REACCIONES Y TIROS EN CIMA
    df_reacciones = estructura_poo.calcular_reacciones_tiros_cima(
        nodo_apoyo="BASE", 
        nodo_cima="HG1"  
    )
    
    # 2. IMPRIMIR TABLA RESUMEN
    estructura_poo.imprimir_reacciones_tiros(mostrar_c2=True)
    
    # 3. GENERAR DIAGRAMAS
    estructura_poo.diagrama_polar_tiros()
    estructura_poo.diagrama_barras_tiros()
    
    # 4. GUARDAR RESULTADOS
    estructura_poo.guardar_reacciones_csv(folder)
    
    # 5. ACCEDER A LOS DATOS
    print(f"\n📋 Datos disponibles:")
    print(f"   - DataFrame: estructura_poo.df_reacciones")
    print(f"   - Diccionario: estructura_poo.estructura_poo.resultados_reacciones")  


if __name__ == "__main__":
    main()

In [ ]:
### 006: POSTES DE HORMIGÓN
postes = PostesHormigon()
resultados_postes = postes.calcular_seleccion_postes(
    estructura_poo=estructura_poo,
    FORZAR_N_POSTES=FORZAR_N_POSTES,
    FORZAR_ORIENTACION=FORZAR_ORIENTACION,
    ANCHO_CRUCETA=ANCHO_CRUCETA,
    PRIORIDAD_DIMENSIONADO=PRIORIDAD_DIMENSIONADO  # Pasar el parámetro
)
postes.imprimir_desarrollo_seleccion_postes()

In [ ]:
# CELDA 200: IMPORTS Y FUNCIONES AUXILIARES
# ============================================================

# IMPORTAR LIBRERÍAS

import numpy as np
import openseespy.opensees as ops
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm as cm
from collections import defaultdict
import math
import random
from matplotlib.patches import Patch

# ============================================================
# FUNCIÓN PARA DETERMINAR EL TIPO DE RESTRICCIÓN DE UN NODO
# ============================================================

def obtener_tipo_restriccion(nodo_nombre, nodes_restrictions, RESTRAINT_TYPES):
    """
    Determina el tipo de restricción de un nodo basado en nodes_restrictions
    """
    if nodo_nombre in nodes_restrictions:
        restriccion = nodes_restrictions[nodo_nombre]
        for tipo, patron in RESTRAINT_TYPES.items():
            if restriccion == patron:
                return tipo
    return 'FREE'  # Por defecto si no está especificado

# ============================================================
# FUNCIÓN PARA SUBDIVIDIR UN SEGMENTO ENTRE DOS NODOS
# ============================================================

# IDENTIFICACIÓN Y SUBDIVISIÓN DE NODOS EN TRAMOS
# ============================================================

def subdividir_segmento(nodo_inicio, nodo_fin, num_subdivisiones, tag_base, node_tags, nodes_key):
    """
    Versión modificada que solo calcula coordenadas y actualiza diccionarios
    """
    if num_subdivisiones <= 1:
        # No se necesitan subdivisiones
        tag_inicio = node_tags[nodo_inicio]
        tag_fin = node_tags[nodo_fin]
        return ([tag_inicio, tag_fin], [nodo_inicio, nodo_fin])
    
    # Obtener coordenadas
    coords_inicio = np.array(nodes_key[nodo_inicio])
    coords_fin = np.array(nodes_key[nodo_fin])
    
    # Crear nodos intermedios
    tags = [node_tags[nodo_inicio]]
    nombres = [nodo_inicio]
    
    for i in range(1, num_subdivisiones):
        # Calcular posición intermedia
        factor = i / num_subdivisiones
        coords_intermedio = coords_inicio + factor * (coords_fin - coords_inicio)
        
        # Crear nombre y tag para el nodo intermedio
        nombre_intermedio = f"{nodo_inicio}_{nodo_fin}_{i}"
        tag_intermedio = tag_base + i
        
        # SOLO ACTUALIZAR DICCIONARIOS, 
        tags.append(tag_intermedio)
        nombres.append(nombre_intermedio)
        
        # Guardar en el diccionario global
        nodes_key[nombre_intermedio] = coords_intermedio.tolist()
        node_tags[nombre_intermedio] = tag_intermedio
    
    tags.append(node_tags[nodo_fin])
    nombres.append(nodo_fin)
    
    return tags, nombres

# ============================================================
# FUNCIÓN PARA ENCONTRAR NODOS INTERMEDIOS EN UN TRAMO
# ============================================================

def encontrar_nodos_en_linea(nodoA, nodoB, todos_nodos, tolerancia=0.1):
    """
    Versión más robusta que funciona independientemente del orden.
    Encuentra todos los nodos que están en la línea entre A y B.
    
    Args:
        nodoA, nodoB (str): Nodos que definen la línea
        todos_nodos (dict): Diccionario de coordenadas
        tolerancia (float): Tolerancia para considerar que está en la línea
    
    Returns:
        list: Nodos ordenados por distancia a A
    """
    coordsA = np.array(todos_nodos[nodoA])
    coordsB = np.array(todos_nodos[nodoB])
    
    # Vector AB y distancia
    vectorAB = coordsB - coordsA
    distAB = np.linalg.norm(vectorAB)
    
    if distAB < 1e-6:
        return [nodoA, nodoB]
    
    # Encontrar todos los nodos en la línea AB
    nodos_en_linea = []
    
    for nombre, coords in todos_nodos.items():
        if nombre == nodoA or nombre == nodoB:
            continue
            
        coordsN = np.array(coords)
        vectorAN = coordsN - coordsA
        
        # Calcular parámetro t (0 en A, 1 en B)
        if distAB > 0:
            t = np.dot(vectorAN, vectorAB) / (distAB * distAB)
        else:
            t = 0
        
        # Verificar si está en el segmento AB (con tolerancia)
        if -tolerancia/distAB <= t <= 1 + tolerancia/distAB:
            # Punto proyectado
            punto_proyectado = coordsA + t * vectorAB
            dist_perpendicular = np.linalg.norm(coordsN - punto_proyectado)
            
            if dist_perpendicular < tolerancia:
                # Calcular distancia real a A
                dist_a_A = np.linalg.norm(coordsN - coordsA)
                nodos_en_linea.append((nombre, dist_a_A, t))
    
    # Ordenar por distancia a A
    nodos_en_linea.sort(key=lambda x: x[1])
    
    # Construir lista final
    resultado = [nodoA]
    for nodo in nodos_en_linea:
        resultado.append(nodo[0])
    resultado.append(nodoB)
    
    return resultado

# ============================================================
# FUNCIÓN AUXILIAR PARA VISUALIZAR RESTRICCIONES
# ============================================================

def restraints_to_symbol(restraints):
    symbols = []
    for r in restraints:
        symbols.append('●' if r == 1 else '◦')
    return f"[{''.join(symbols)}]"

# FUNCIONES PARA CÁLCULO DE PROPIEDADES
# ============================================================

def calcular_propiedades_ha_exactas(diametro, espesor, cant_hierros, diametro_hierro, recubrimiento, 
                                   n, factor_I_fisurado, factor_E_efectivo, E_hormigon, nu_hormigon):
    """
    CORRECCIÓN 1 & 2: Calcula propiedades EXACTAS de secciones de hormigón armado
    considerando la variación no lineal (∝ D⁴) y homogeneización correcta
    
    Args:
        n: Relación de módulos E_acero/E_hormigon
        factor_I_fisurado: Factor de reducción por fisuración
        factor_E_efectivo: Factor de reducción del módulo elástico
        E_hormigon: Módulo de elasticidad del hormigón
        nu_hormigon: Coeficiente de Poisson del hormigón
    """
    # Diámetro interior
    diametro_int = diametro - 2 * espesor
    
    # Verificar que el espesor sea válido
    if diametro_int <= 0:
        diametro_int = 0.01  # Mínimo diámetro interior para evitar errores
    
    # HORMIGÓN: Propiedades exactas (∝ D⁴)
    A_h = (np.pi / 4) * (diametro**2 - diametro_int**2)
    I_h = (np.pi / 64) * (diametro**4 - diametro_int**4)  # ∝ D⁴
    J_h = (np.pi / 32) * (diametro**4 - diametro_int**4)  # ∝ D⁴
    
    # ACERO: Considerando recubrimiento y posición real
    radio_centro_acero = diametro/2 - recubrimiento - diametro_hierro/2
    area_barra = (np.pi / 4) * diametro_hierro**2
    A_a = cant_hierros * area_barra
    
    # Inercia del acero usando teorema de Steiner (barras distribuidas uniformemente)
    I_a = cant_hierros * area_barra * radio_centro_acero**2
    
    # CORRECCIÓN 2: HOMOGENEIZACIÓN CORRECTA (A_homog = A_h + (n-1)×A_a)
    A_homog = A_h + (n - 1) * A_a
    I_homog = I_h + (n - 1) * I_a
    
    # Torsión: simplificación conservadora (solo hormigón)
    J_homog = J_h
    
    # Aplicar factores de reducción por fisuración
    A_eff = A_homog
    I_eff = I_homog * factor_I_fisurado
    E_eff = E_hormigon * factor_E_efectivo
    G_eff = E_eff / (2 * (1 + nu_hormigon))
    J_eff = J_homog * factor_I_fisurado
    
    return {
        # Propiedades geométricas
        'diametro': diametro,
        'espesor': espesor,
        'diametro_int': diametro_int,
        'recubrimiento': recubrimiento,
        'radio_acero': radio_centro_acero,
        
        # Propiedades materiales
        'A_hormigon': A_h,
        'I_hormigon': I_h,
        'J_hormigon': J_h,
        'A_acero': A_a,
        'I_acero': I_a,
        
        # Propiedades homogeneizadas
        'A_homog': A_homog,
        'I_homog': I_homog,
        'J_homog': J_homog,
        
        # Propiedades efectivas (con fisuración)
        'A_eff': A_eff,
        'I_eff': I_eff,
        'J_eff': J_eff,
        'E_eff': E_eff,
        'G_eff': G_eff,
        
        # Información de armadura
        'cant_hierros': cant_hierros,
        'diametro_hierro': diametro_hierro
    }

def interpolar_dimensiones_y_recalcular(prop_inicio, prop_fin, factor, propiedades_tramo,
                                       n, factor_I_fisurado, factor_E_efectivo, E_hormigon, nu_hormigon):
    """
    CORRECCIÓN 1: Interpola DIMENSIONES y luego RECALCULA propiedades exactas
    Evita la interpolación lineal incorrecta de propiedades ∝ D⁴
    """
    # Interpolar dimensiones geométricas linealmente
    diametro_interp = prop_inicio['diametro'] + factor * (prop_fin['diametro'] - prop_inicio['diametro'])
    espesor_interp = prop_inicio['espesor'] + factor * (prop_fin['espesor'] - prop_inicio['espesor'])
    
    # Mantener constantes los parámetros de armadura y recubrimiento
    cant_hierros = propiedades_tramo['cant_hierros']
    diametro_hierro = propiedades_tramo['diametro_hierro']
    recubrimiento = propiedades_tramo.get('recubrimiento', 0.04)
    
    # CORRECCIÓN 1: RECALCULAR propiedades exactas con dimensiones interpoladas
    return calcular_propiedades_ha_exactas(
        diametro_interp, espesor_interp, cant_hierros, diametro_hierro, recubrimiento,
        n, factor_I_fisurado, factor_E_efectivo, E_hormigon, nu_hormigon
    )

def crear_diccionario_elementos(tramos_propiedades, nodes_final, nodes_key):
    """
    Crea un diccionario de elementos con propiedades interpoladas exactas
    para cada segmento entre nodos consecutivos en cada tramo
    """
    global n, factor_I_fisurado, factor_E_efectivo, E_hormigon, nu_hormigon

    elementos_dict = {}
    element_counter = 1
    
    # Crear un diccionario inverso: nombre_nodo -> tag
    nombre_a_tag = {info['nombre']: tag for tag, info in nodes_final.items()}
    
    for tramo_nombre, propiedades in tramos_propiedades.items():
        print(f"📐 Procesando tramo: {tramo_nombre}")
        
        # Obtener nodos extremos del tramo
        nodo_inicio_nombre = propiedades['nodos'][0]
        nodo_fin_nombre = propiedades['nodos'][1]
        
        # Encontrar todos los nodos en este tramo (ordenados)
        nodos_tramo = encontrar_nodos_en_linea(
            nodo_inicio_nombre, nodo_fin_nombre, nodes_key
        )
        
        print(f"   Nodos en tramo ({len(nodos_tramo)}): {nodos_tramo}")
        
        # Calcular propiedades en los extremos del tramo
        prop_inicio = calcular_propiedades_ha_exactas(
            propiedades['diametro_inicial'],
            propiedades['espesor_inicial'],
            propiedades['cant_hierros'],
            propiedades['diametro_hierro'],
            propiedades['recubrimiento'],
            n, factor_I_fisurado, factor_E_efectivo, E_hormigon, nu_hormigon
        )
        
        prop_fin = calcular_propiedades_ha_exactas(
            propiedades['diametro_final'],
            propiedades['espesor_final'],
            propiedades['cant_hierros'],
            propiedades['diametro_hierro'],
            propiedades['recubrimiento'],
            n, factor_I_fisurado, factor_E_efectivo, E_hormigon, nu_hormigon
        )
        
        # Crear elementos entre nodos consecutivos
        for i in range(len(nodos_tramo) - 1):
            nodoA_nombre = nodos_tramo[i]
            nodoB_nombre = nodos_tramo[i + 1]
            
            # Obtener tags de los nodos
            nodoA_tag = nombre_a_tag[nodoA_nombre]
            nodoB_tag = nombre_a_tag[nodoB_nombre]
            
            # Calcular factor de interpolación (posición relativa en el tramo)
            coordsA = np.array(nodes_key[nodoA_nombre])
            coordsB = np.array(nodes_key[nodoB_nombre])
            coords_inicio = np.array(nodes_key[nodo_inicio_nombre])
            coords_fin = np.array(nodes_key[nodo_fin_nombre])
            
            # Distancia desde el inicio del tramo hasta el punto medio del elemento
            punto_medio = (coordsA + coordsB) / 2
            vector_tramo = coords_fin - coords_inicio
            longitud_tramo = np.linalg.norm(vector_tramo)
            
            if longitud_tramo > 0:
                vector_a_medio = punto_medio - coords_inicio
                # Proyección escalar a lo largo del tramo
                factor = np.dot(vector_a_medio, vector_tramo) / (longitud_tramo ** 2)
                factor = max(0, min(1, factor))  # Limitar entre 0 y 1
            else:
                factor = 0.5
            
        # Interpolar propiedades EXACTAS en el punto medio del elemento (PASANDO LOS PARÁMETROS)
            propiedades_elemento = interpolar_dimensiones_y_recalcular(
                prop_inicio, prop_fin, factor, propiedades,
                n, factor_I_fisurado, factor_E_efectivo, E_hormigon, nu_hormigon
            )
            
            # Calcular longitud del elemento
            longitud_elemento = np.linalg.norm(coordsB - coordsA)
            
            # Crear entrada del elemento
            elemento_info = {
                'id': element_counter,
                'tramo': tramo_nombre,
                'nodos': [nodoA_tag, nodoB_tag],
                'nombres_nodos': [nodoA_nombre, nodoB_nombre],
                'posicion_relativa': factor,
                'longitud': longitud_elemento,
                'propiedades': propiedades_elemento,
                'datos_Solver': {
                    'A': propiedades_elemento['A_eff'],
                    'E': propiedades_elemento['E_eff'],
                    'G': propiedades_elemento['G_eff'],
                    'Ix': propiedades_elemento['I_eff'],  # Iy (flexión local y)
                    'Iy': propiedades_elemento['I_eff'],  # Iz (flexión local z) 
                    'Iz': propiedades_elemento['J_eff'],  # J (torsión)
                    'tipo': 'elasticBeamColumn'
                }
            }
            
            elementos_dict[element_counter] = elemento_info
            element_counter += 1
    
    return elementos_dict

# GENERAR COLORES BIEN DISTINTOS
def generar_colores_distintos(num_colores):
    """Genera colores bien distintos usando distribución uniforme en HSV"""
    colores = []
    
    # Si solo necesitamos 1-2 colores, usar valores fijos muy distintos
    if num_colores == 1:
        return [(0.0, 0.5, 0.8)]  # Azul
    elif num_colores == 2:
        return [(0.0, 0.5, 0.8), (0.8, 0.5, 0.8)]  # Azul y rojo
    
    # Para 3 o más colores, distribuir uniformemente en el círculo HSV
    for i in range(num_colores):
        # Distribuir uniformemente el tono (0-1)
        hue = i / num_colores
        
        # Variar ligeramente la saturación y el valor para evitar colores demasiado similares
        saturation = 0.7 + random.uniform(-0.1, 0.1)  # 0.6-0.8
        value = 0.8 + random.uniform(-0.1, 0.1)       # 0.7-0.9
        
        # Asegurarse de que los valores estén en rango
        saturation = max(0.6, min(0.9, saturation))
        value = max(0.7, min(0.95, value))
        
        # Convertir HSV a RGB
        h_i = math.floor(hue * 6)
        f = hue * 6 - h_i
        p = value * (1 - saturation)
        q = value * (1 - f * saturation)
        t = value * (1 - (1 - f) * saturation)
        
        if h_i == 0:
            r, g, b = value, t, p
        elif h_i == 1:
            r, g, b = q, value, p
        elif h_i == 2:
            r, g, b = p, value, t
        elif h_i == 3:
            r, g, b = p, q, value
        elif h_i == 4:
            r, g, b = t, p, value
        else:
            r, g, b = value, p, q
            
        colores.append((r, g, b))
    
    return colores

# VERIFICAR Y AJUSTAR COLORES PARA MAXIMA DISTINCIÓN
def ajustar_colores_para_distincion(colores, umbral_minimo=0.3):
    """
    Ajusta colores para asegurar que sean suficientemente distintos
    umbral_minimo: distancia mínima en espacio RGB (0-1)
    """
    def distancia_color(c1, c2):
        return math.sqrt((c1[0]-c2[0])**2 + (c1[1]-c2[1])**2 + (c1[2]-c2[2])**2)
    
    colores_ajustados = colores.copy()
    
    for i in range(len(colores_ajustados)):
        for j in range(i + 1, len(colores_ajustados)):
            dist = distancia_color(colores_ajustados[i], colores_ajustados[j])
            intentos = 0
            while dist < umbral_minimo and intentos < 10:
                # Ajustar el color j
                nuevo_color = (
                    random.random(),
                    random.random(), 
                    random.random()
                )
                # Verificar distancia con todos los colores existentes
                dist_min = min(distancia_color(nuevo_color, colores_ajustados[k]) 
                             for k in range(len(colores_ajustados)) if k != j)
                if dist_min >= umbral_minimo:
                    colores_ajustados[j] = nuevo_color
                    dist = distancia_color(colores_ajustados[i], nuevo_color)
                intentos += 1
                
    return colores_ajustados

# Helper: normalize coordinates to floats and a tuple rounded to tolerance
def coord_tuple(node_info, tol=1e-9):
    c = node_info.get('coordenadas', {})
    x = float(c.get('x', 0.0))
    y = float(c.get('y', 0.0))
    z = float(c.get('z', 0.0))
    # Round to avoid floating precision duplicates
    return (round(x,9), round(y,9), round(z,9))



def is_node_fixed(node_info):
    """CORRECCIÓN: Verifica si un nodo está completamente empotrado"""
    dof = node_info.get('dof', [0,0,0,0,0,0])
    # Fixed si TODOS los DOF están restringidos (1) - no solo traslaciones
    return all(dof)  


def comparar_hipotesis(resultados_dict, parametro='desplazamiento_max'):
    """Compara un parámetro entre todas las hipótesis"""
    comparacion = {}
    
    for hipotesis, resultado in resultados_dict.items():
        if resultado['analisis_exitoso']:
            if parametro == 'desplazamiento_max':
                comparacion[hipotesis] = resultado['valores_maximos']['desplazamiento_max']
            elif parametro == 'momento_max':
                comparacion[hipotesis] = resultado['valores_maximos']['momento_max']
            elif parametro == 'axial_max':
                comparacion[hipotesis] = resultado['valores_maximos']['axial_max']
            elif parametro == 'corte_max':
                comparacion[hipotesis] = resultado['valores_maximos']['corte_max']
    
    return comparacion


In [ ]:
# CELDA 201: PARÁMETROS (INPUTS)
# ============================================================

# Valores en X Y Z NODOS ORIGINALES

# ESTRUCTURA S ADT-NH2 : bS - Piquete 20: altura 23,9m / 
nodes_key = nodes_key

# CARGAS 
# ============================================================
cargas_key = cargas_key
mult = 10
 # Convertir todas las cargas a N
for hipotesis in cargas_key:
    for nodo in cargas_key[hipotesis]:
        # Multiplicar cada componente [X, Y, Z] por mult
        cargas_key[hipotesis][nodo] = [
            cargas_key[hipotesis][nodo][0] * mult,  # Componente X
            cargas_key[hipotesis][nodo][1] * mult,  # Componente Y
            cargas_key[hipotesis][nodo][2] * mult   # Componente Z
        ]

print(f"✅ Todas las cargas en cargas_key han sido multiplicadas por {mult}")

 # Hipótesis activas para análisis
HIPOTESIS_ACTIVAS = list(cargas_key.keys())     # todas

# Tipos de restricciones
RESTRAINT_TYPES = {
    'FIXED': [1, 1, 1, 1, 1, 1],      # Empotramiento completo
    'HINGED': [1, 1, 1, 0, 0, 0],     # Articulación (solo traslaciones)
    'ROLLER_X': [0, 1, 1, 0, 0, 0],   # Rodillo en dirección X
    'ROLLER_Y': [1, 0, 1, 0, 0, 0],   # Rodillo en dirección Y
    'ROLLER_Z': [1, 1, 0, 0, 0, 0],   # Rodillo en dirección Z
    'FREE': [0, 0, 0, 0, 0, 0]        # Libre
}
# Agregar BASE2 para tener 2 fijos
if 'BASE' in nodes_key:
    # Obtener coordenadas actuales de BASE
    x, y, z = nodes_key['BASE']
    
    # Crear nuevo nodo BASE2 con Z incrementado
    nodes_key['BASE2'] = (x, y, z + 0.01)
else:
    print("Error: El nodo BASE no existe en nodes_key")

# Restricciones de nodos: movimiento x y z | rotación x y z
nodes_restrictions  = {
    # Empotramiento completo en bases (6 GDL restringidos)
    'BASE': RESTRAINT_TYPES['FIXED'],
    'BASE2': RESTRAINT_TYPES['FIXED'],
    #'B2': RESTRAINT_TYPES['FIXED'],
}

# DEFINICIÓN DE TRAMOS Y SUBDIVISIONES
# ============================================================

subdiv_1 = 30
subdiv_2 = 20
subdiv_3 = 15


# Diccionario de tramos: {nombre_tramo: [nodo_inicial, nodo_final]}
tramos_propiedades = {
    "COL1": {                  # DESDE SUELO HASTA CABLE GUARDIA O TOP
        "nodos": [
            "BASE",
            "HG1" if (TERNA == "Doble" and CANT_HG == 1) else "TOP"
        ],
        "subdividir": 20,
        "diametro_inicial": 0.664,
        "espesor_inicial": 0.11,
        "diametro_final": 0.415,
        "espesor_final": 0.09,
        "cant_hierros": 17,
        "diametro_hierro": 0.015,
        "recubrimiento": 0.04
    },
    "MEN1": {
        "nodos": [
            "CROSS_H1",
            "C1_L"
        ],
        "subdividir": 5,
        "diametro_inicial": 0.28,
        "espesor_inicial": 0.09,
        "diametro_final": 0.28,
        "espesor_final": 0.09,
        "cant_hierros": 10,
        "diametro_hierro": 0.015,
        "recubrimiento": 0.04
    },
    "MEN2": {
        "nodos": [
            "CROSS_H1",
            "C1_R"
        ],
        "subdividir": 5,
        "diametro_inicial": 0.28,
        "espesor_inicial": 0.09,
        "diametro_final": 0.28,
        "espesor_final": 0.09,
        "cant_hierros": 10,
        "diametro_hierro": 0.015,
        "recubrimiento": 0.04
    },
    "MEN3": {
        "nodos": [
            "CROSS_H2",
            "C2_L"
        ],
        "subdividir": 5,
        "diametro_inicial": 0.28,
        "espesor_inicial": 0.09,
        "diametro_final": 0.28,
        "espesor_final": 0.09,
        "cant_hierros": 10,
        "diametro_hierro": 0.015,
        "recubrimiento": 0.04
    },
    "MEN4": {
        "nodos": [
            "CROSS_H2",
            "C2_R"
        ],
        "subdividir": 5,
        "diametro_inicial": 0.28,
        "espesor_inicial": 0.09,
        "diametro_final": 0.28,
        "espesor_final": 0.09,
        "cant_hierros": 10,
        "diametro_hierro": 0.015,
        "recubrimiento": 0.04
    },
    "MEN5": {
        "nodos": [
            "CROSS_H3",
            "C3_L"
        ],
        "subdividir": 5,
        "diametro_inicial": 0.28,
        "espesor_inicial": 0.09,
        "diametro_final": 0.28,
        "espesor_final": 0.09,
        "cant_hierros": 10,
        "diametro_hierro": 0.015,
        "recubrimiento": 0.04
    },
    "MEN6": {
        "nodos": [
            "CROSS_H3",
            "C3_R"
        ],
        "subdividir": 5,
        "diametro_inicial": 0.28,
        "espesor_inicial": 0.09,
        "diametro_final": 0.28,
        "espesor_final": 0.09,
        "cant_hierros": 10,
        "diametro_hierro": 0.015,
        "recubrimiento": 0.04
    },}

# Valor por defecto para subdivisiones
default_subdivisions = 4

# ============================================================
# PARÁMETROS DE MATERIALES 
# ============================================================

# Propiedades del hormigón (H30)
E_hormigon = 25e9        # Módulo de elasticidad del hormigón (Pa)
nu_hormigon = 0.2        # Coeficiente de Poisson
fck = 30e6               # Resistencia característica del hormigón (Pa)
fct = 0.3 * fck**0.67    # Resistencia a tracción (approx.)
densidad_hormigon = 25000  # Peso específico del hormigón armado (N/m³) - AGREGADO


# Propiedades del acero (ADN 500)
E_acero = 200e9          # Módulo de elasticidad del acero (Pa)
fy = 500e6               # Límite elástico del acero (Pa)

# Relación de módulos CORREGIDA
n = E_acero / E_hormigon  # n ≈ 8

# Factores de reducción por fisuración (normativas)
factor_I_fisurado = 0.5   # típico 0.3-0.7 para HA fisurado
factor_E_efectivo = 0.8   # reducción por fluencia

print("🔧 PARÁMETROS DE MATERIALES:")
print(f"   Hormigón H30: E = {E_hormigon/1e9:.1f} GPa, fck = {fck/1e6:.1f} MPa, γ = {densidad_hormigon} N/m³")
print(f"   Acero ADN500: E = {E_acero/1e9:.1f} GPa, fy = {fy/1e6:.1f} MPa")
print(f"   Relación n = E_acero/E_hormigon = {n:.1f}")
print(f"   Factores de reducción: I_fisurado = {factor_I_fisurado}, E_efectivo = {factor_E_efectivo}")


# DEFINICIONES PARA FUNCIONES POSTERIORES
NODOS_ORIGINALES = list(nodes_key.keys())  # Todos los nodos definidos inicialmente
NODOS_ETIQUETAR = NODOS_ORIGINALES  # O definir una lista específica si se prefiere
PUNTOS_CLAVE = NODOS_ORIGINALES  # Para análisis de resultados


In [ ]:
# CELDA 202: FUNCION PRINCIPAL DE ANALISIS LINEAR

def realizar_analisis_por_hipotesis(hipotesis_nombre, cargas_hipotesis, elementos_dict, nodes_final, PUNTOS_CLAVE):
    """
    VERSIÓN CORREGIDA - Transformaciones geométricas según ejemplo oficial
    """
    print(f"\n🎯 INICIANDO ANÁLISIS PARA HIPÓTESIS: {hipotesis_nombre}")
    print("=" * 60)

    """# DIAGNÓSTICO TEMPORAL
    print(f"🔍 DIAGNÓSTICO DE CARGAS PARA {hipotesis_nombre}:")
    cargas_no_zero = {nodo: carga for nodo, carga in cargas_hipotesis.items() if any(val != 0 for val in carga)}
    print(f"   Cargas no cero: {len(cargas_no_zero)}")"""

    # Limpiar modelo existente
    ops.wipe()
    
    # Configuración del modelo
    ops.model('basic', '-ndm', 3, '-ndf', 6)
    
    # DEFINICIÓN DE NODOS
    nodos_completos_tags = set()
    for elem in elementos_dict.values():
        nodos_completos_tags.update(elem['nodos'])
    
    for tag in nodos_completos_tags:
        node_info = nodes_final[tag]
        coords = node_info['coordenadas']
        ops.node(tag, coords['x'], coords['y'], coords['z'])
        ops.fix(tag, *node_info['dof'])
    
    # DEFINICIÓN DE ELEMENTOS - CORRECCIÓN SEGÚN EJEMPLO OFICIAL
    # ✅ CORRECCIÓN: Transformaciones geométricas como en el ejemplo demo
    
    # Crear múltiples transformaciones para diferentes orientaciones
    transfTag_z = 1    # Para elementos verticales (dirección Z)
    transfTag_x = 2    # Para elementos en dirección X
    transfTag_y = 3    # Para elementos en dirección Y
    transfTag_xy = 4   # Para elementos diagonales en plano XY
    transfTag_xz = 5   # Para elementos diagonales en plano XZ
    
    # Definir transformaciones según ejemplo oficial
    # vecxz define el plano local x-z del elemento
    ops.geomTransf('Linear', transfTag_z, 0., 1., 0.)   # Como en demo: verticales
    ops.geomTransf('Linear', transfTag_x, 0., 1., 0.)   # Como en demo: dirección X  
    ops.geomTransf('Linear', transfTag_y, 1., 0., 0.)    # Como en demo: dirección Y
    ops.geomTransf('Linear', transfTag_xy, 0., 0., 1.)   # Diagonal XY: usar Z como referencia
    ops.geomTransf('Linear', transfTag_xz, 0., 1., 0.)  # Diagonal XZ: igual que verticales
    
    for eid, elem in elementos_dict.items():
        nodoA, nodoB = elem['nodos']
        propiedades = elem['propiedades']
        
        coordsA = nodes_final[nodoA]['coordenadas']
        coordsB = nodes_final[nodoB]['coordenadas']
        
        # Calcular vector de dirección del elemento
        dx = coordsB['x'] - coordsA['x']
        dy = coordsB['y'] - coordsA['y'] 
        dz = coordsB['z'] - coordsA['z']
        
        # ✅ CORRECCIÓN: Determinar transformación basado en la dirección predominante
        # Similar al criterio del ejemplo oficial
        
        # Calcular magnitudes absolutas
        abs_dx = abs(dx)
        abs_dy = abs(dy) 
        abs_dz = abs(dz)
        
        # Determinar dirección predominante
        if abs_dz > max(abs_dx, abs_dy) * 2:  # Elemento predominantemente vertical
            transfTag = transfTag_z
            orientacion = "VERTICAL (Z)"
        elif abs_dx > max(abs_dy, abs_dz) * 2:  # Elemento predominantemente en X
            transfTag = transfTag_x
            orientacion = "HORIZONTAL X"
        elif abs_dy > max(abs_dx, abs_dz) * 2:  # Elemento predominantemente en Y
            transfTag = transfTag_y  
            orientacion = "HORIZONTAL Y"
        elif abs_dx > 0 and abs_dy > 0 and abs_dz < 0.1:  # Elemento en plano XY
            transfTag = transfTag_xy
            orientacion = "DIAGONAL XY"
        else:  # Elemento diagonal en XZ o general
            transfTag = transfTag_xz
            orientacion = "DIAGONAL/OTRO"
        
        A = propiedades['A_eff']
        E = propiedades['E_eff']
        G = propiedades['G_eff']
        Iy = propiedades['I_eff']
        Iz = propiedades['I_eff']
        J = propiedades['J_eff']
        
        """# Debug info
        print(f"   Elemento {eid}: {orientacion} | dx={dx:.2f}, dy={dy:.2f}, dz={dz:.2f} | transfTag={transfTag}")"""
        
        # Verificar propiedades
        if A < 1e-9 or E < 1e-6:
            print(f"   ⚠️ Propiedades pequeñas: A={A:.2e}, E={E:.2e}")
        
        try:
            ops.element('elasticBeamColumn', eid, nodoA, nodoB, A, E, G, J, Iy, Iz, transfTag)
        except Exception as e:
            print(f"❌ Error creando elemento {eid}: {e}")
            return {
                'nombre': hipotesis_nombre,
                'analisis_exitoso': False,
                'error': f"Error en elemento {eid}: {e}"
            }
    
    # ✅ CORRECCIÓN: APLICAR PESO PROPIO COMO CARGAS NODALES (MÉTODO MÁS SEGURO)
    print("📦 Aplicando peso propio como cargas nodales...")
    
    # 1. TimeSeries y Pattern único para todas las cargas
    ops.timeSeries('Constant', 1)
    ops.pattern('Plain', 1, 1)
    
    # Calcular peso acumulado por nodo
    from collections import defaultdict
    peso_nodal = defaultdict(float)
    peso_propio_total = 0
    
    for eid, elem in elementos_dict.items():
        A_eff = elem['propiedades']['A_eff']
        L = elem['longitud']
        peso_elemento = densidad_hormigon * A_eff * L
        
        # Distribuir peso a los nodos (50% cada uno)
        nodoA, nodoB = elem['nodos']
        peso_nodal[nodoA] += peso_elemento / 2.0
        peso_nodal[nodoB] += peso_elemento / 2.0
        peso_propio_total += peso_elemento
    
    # Aplicar cargas nodales de peso propio (dirección Z global negativa)
    for nodo_tag, peso in peso_nodal.items():
        ops.load(nodo_tag, 0, 0, -peso, 0, 0, 0)  # Solo en Z global
    
    print(f"   Peso propio total aplicado: {peso_propio_total/1000:.1f} kN")
    
    # 2. APLICAR CARGAS EXTERNAS DE LA HIPÓTESIS
    cargas_aplicadas = {}
    total_carga = [0, 0, 0]
    
    for nombre_nodo, carga in cargas_hipotesis.items():
        if any(val != 0 for val in carga):
            nodo_tag = None
            for tag, info in nodes_final.items():
                if info['nombre'] == nombre_nodo:
                    nodo_tag = tag
                    break
            
            if nodo_tag is not None:
                ops.load(nodo_tag, carga[0], carga[1], carga[2], 0, 0, 0)
                cargas_aplicadas[nombre_nodo] = carga
                total_carga[0] += carga[0]
                total_carga[1] += carga[1]
                total_carga[2] += carga[2]
                print(f"   ✅ Carga en {nombre_nodo} (tag {nodo_tag}): {carga}")
            else:
                print(f"   ❌ Nodo {nombre_nodo} no encontrado")
    
    print(f"   Cargas externas aplicadas: {len(cargas_aplicadas)} nodos")
    
    # 3. CONFIGURACIÓN DEL ANÁLISIS (como en el ejemplo oficial)
    ops.constraints('Plain')
    ops.numberer('RCM')
    ops.system('BandGeneral')
    ops.test('NormDispIncr', 1.0e-6, 10)
    ops.algorithm('Linear')  # Usar Linear primero como en el ejemplo
    ops.integrator('LoadControl', 1.0)
    ops.analysis('Static')
    
    # 4. EJECUTAR ANÁLISIS
    analisis_exitoso = False
    try:
        print("🔍 Realizando análisis estático...")
        resultado = ops.analyze(1)
        analisis_exitoso = (resultado == 0)
        
        if analisis_exitoso:
            print("✅ Análisis completado exitosamente")
        else:
            print("❌ Falló el análisis estático")
            # Intentar con algoritmo Newton
            print("🔄 Intentando con algoritmo Newton...")
            ops.wipeAnalysis()
            ops.algorithm('Newton')
            ops.analysis('Static')
            resultado = ops.analyze(1)
            analisis_exitoso = (resultado == 0)
            
    except Exception as e:
        print(f"❌ Error en análisis: {e}")
        analisis_exitoso = False
    
    # [El resto del código para recopilar resultados permanece igual...]
    
    # RECOPILAR RESULTADOS (incluyendo contribución del peso propio y cargas adicionales)
    resultados_hipotesis = {
        'nombre': hipotesis_nombre,
        'cargas_aplicadas': cargas_aplicadas,
        'total_carga': total_carga,
        'peso_propio_total': peso_propio_total,
        'hay_cargas_adicionales': False,
        'analisis_exitoso': analisis_exitoso,
        'reacciones': {},
        'desplazamientos': {},
        'fuerzas_internas': {},
        'valores_maximos': {}
    }
    
    if analisis_exitoso:
        ops.reactions()
        
        # REACCIONES EN NODOS FIJOS (incluyen peso propio + cargas adicionales)
        reacciones_totales = [0, 0, 0, 0, 0, 0]
        for tag in nodos_completos_tags:
            node_info = nodes_final[tag]
            if node_info['tipo_restriccion'] == 'FIXED':
                reacciones = ops.nodeReaction(tag)
                resultados_hipotesis['reacciones'][tag] = reacciones
                for i in range(6):
                    reacciones_totales[i] += reacciones[i]
        
        resultados_hipotesis['reacciones_totales'] = reacciones_totales
        
        # DESPLAZAMIENTOS EN TODOS LOS NODOS
        desplazamientos_totales = []
        for tag in nodos_completos_tags:
            desplazamientos = ops.nodeDisp(tag)
            resultados_hipotesis['desplazamientos'][tag] = desplazamientos
            desp_total = np.sqrt(desplazamientos[0]**2 + desplazamientos[1]**2 + desplazamientos[2]**2)
            desplazamientos_totales.append(desp_total)
        
        # FUERZAS INTERNAS EN TODOS LOS ELEMENTOS
        momentos = []
        fuerzas_axiales = []
        fuerzas_corte = []
        
        for eid in elementos_dict.keys():
            fuerzas = ops.eleForce(eid)
            resultados_hipotesis['fuerzas_internas'][eid] = fuerzas
            
            momento_max = max(abs(fuerzas[4]), abs(fuerzas[5]), abs(fuerzas[10]), abs(fuerzas[11]))
            axial_max = abs(fuerzas[0])
            corte_max = max(abs(fuerzas[1]), abs(fuerzas[2]), abs(fuerzas[7]), abs(fuerzas[8]))
            
            momentos.append(momento_max)
            fuerzas_axiales.append(axial_max)
            fuerzas_corte.append(corte_max)
        
        # VALORES MÁXIMOS
        resultados_hipotesis['valores_maximos'] = {
            'desplazamiento_max': max(desplazamientos_totales) * 1000 if desplazamientos_totales else 0,
            'momento_max': max(momentos) if momentos else 0,
            'axial_max': max(fuerzas_axiales) if fuerzas_axiales else 0,
            'corte_max': max(fuerzas_corte) if fuerzas_corte else 0
        }
        
    # VERIFICACIÓN DE EQUILIBRIO
    print("\n📐 VERIFICACIÓN DE EQUILIBRIO:")
    print(f"   Peso propio: {peso_propio_total/1000:.1f} kN (↓, negativo)")
    print(f"   Cargas adicionales: FX={total_carga[0]/1000:.1f}, FY={total_carga[1]/1000:.1f}, FZ={total_carga[2]/1000:.1f} kN")
    print(f"   Reacciones totales: FX={reacciones_totales[0]/1000:.1f}, FY={reacciones_totales[1]/1000:.1f}, FZ={reacciones_totales[2]/1000:.1f} kN")
    
    # Fuerzas aplicadas (negativas para peso propio y cargas hacia abajo)
    fuerzas_aplicadas_z = -peso_propio_total + total_carga[2]  # peso propio es negativo, cargas pueden ser +/-
    
    # Las reacciones son positivas cuando contrarrestan las cargas (hacia arriba)
    reacciones_z = reacciones_totales[2]
    
    # En equilibrio: fuerzas_aplicadas + reacciones ≈ 0
    suma_fuerzas_z = fuerzas_aplicadas_z + reacciones_z
    
    # También verificar en X e Y
    suma_fuerzas_x = total_carga[0] + reacciones_totales[0]
    suma_fuerzas_y = total_carga[1] + reacciones_totales[1]
    
    print(f"   Fuerzas aplicadas netas Z: {fuerzas_aplicadas_z/1000:.2f} kN")
    print(f"   Suma fuerzas X: {suma_fuerzas_x/1000:.2f} kN")
    print(f"   Suma fuerzas Y: {suma_fuerzas_y/1000:.2f} kN") 
    print(f"   Suma fuerzas Z: {suma_fuerzas_z/1000:.2f} kN")
    
    # Tolerancia del 1% de la mayor fuerza aplicada
    fuerza_maxima = max(abs(peso_propio_total), abs(total_carga[2]), abs(reacciones_totales[2]))
    tolerancia = 0.01 * fuerza_maxima if fuerza_maxima > 0 else 1.0
    
    equilibrio_x_ok = abs(suma_fuerzas_x) < tolerancia
    equilibrio_y_ok = abs(suma_fuerzas_y) < tolerancia  
    equilibrio_z_ok = abs(suma_fuerzas_z) < tolerancia
    
    equilibrio_ok = equilibrio_x_ok and equilibrio_y_ok and equilibrio_z_ok
    
    if equilibrio_ok:
        print("✅ EQUILIBRIO: Correcto en todas las direcciones")
    else:
        if not equilibrio_x_ok:
            print(f"⚠️ DESEQUILIBRIO EN X: {suma_fuerzas_x/1000:.3f} kN (tolerancia: ±{tolerancia/1000:.3f} kN)")
        if not equilibrio_y_ok:
            print(f"⚠️ DESEQUILIBRIO EN Y: {suma_fuerzas_y/1000:.3f} kN (tolerancia: ±{tolerancia/1000:.3f} kN)")
        if not equilibrio_z_ok:
            print(f"⚠️ DESEQUILIBRIO EN Z: {suma_fuerzas_z/1000:.3f} kN (tolerancia: ±{tolerancia/1000:.3f} kN)")
    
    # Agregar información de equilibrio a los resultados
    resultados_hipotesis['equilibrio'] = {
        'suma_fuerzas_x': suma_fuerzas_x,
        'suma_fuerzas_y': suma_fuerzas_y, 
        'suma_fuerzas_z': suma_fuerzas_z,
        'equilibrio_ok': equilibrio_ok,
        'tolerancia': tolerancia
    }
    
    return resultados_hipotesis

In [ ]:
# CELDA 203: IDENTIFICACIÓN Y SUBDIVISIÓN DE NODOS EN TRAMOS
# ============================================================

# Inicializar diccionarios globales
node_tags = {}
nodes_final = {}

# Asignar tags únicos a los nodos originales
tag_counter = 1
for node_name in nodes_key:
    node_tags[node_name] = tag_counter
    tag_counter += 1

# Procesar cada tramo para subdividir y crear nodos intermedios 
for tramo_nombre, propiedades in tramos_propiedades.items():
    nodo_inicio = propiedades['nodos'][0]
    nodo_fin = propiedades['nodos'][1]
    num_subdivisiones = propiedades['subdividir']
    
    # Subdividir el tramo
    tags, nombres = subdividir_segmento(
        nodo_inicio, nodo_fin, num_subdivisiones, 
        tag_counter, node_tags, nodes_key
    )
    
    # Actualizar el contador de tags
    tag_counter = max(tags) + 1 if tags else tag_counter
    
    print(f"✅ Tramo {tramo_nombre}: {len(tags)} nodos preparados ")

# Crear el diccionario nodes_final con toda la información
for node_name, tag in node_tags.items():
    # Obtener coordenadas
    coords = nodes_key[node_name]
    
    # Determinar restricciones (DOF)
    tipo_restriccion = obtener_tipo_restriccion(node_name, nodes_restrictions, RESTRAINT_TYPES)
    dof = RESTRAINT_TYPES[tipo_restriccion]
    
    # Determinar si es nodo original o intermedio
    es_original = node_name in nodes_key  # Usa el diccionario original de definición
    
    # Crear entrada en nodes_final
    nodes_final[tag] = {
        'id': tag,
        'nombre': node_name,
        'coordenadas': {
            'x': float(coords[0]),
            'y': float(coords[1]), 
            'z': float(coords[2])
        },
        'dof': dof,
        'tipo_restriccion': tipo_restriccion,
        'es_original': es_original,
        'es_intermedio': not es_original
    }

# Mostrar algunos nodos como ejemplo
print(f"\n🔍 NODOS FINALES:")
for i, (tag, node_info) in enumerate(list(nodes_final.items())):
    dof_symbols = restraints_to_symbol(node_info['dof'])
    print(f"   Nodo {tag}: '{node_info['nombre']}' ({node_info['coordenadas']['x']}, {node_info['coordenadas']['y']}, {node_info['coordenadas']['z']}) {dof_symbols} {'(ORIGINAL)' if node_info['es_original'] else '(INTERMEDIO)'}")

print(f"\n📊 RESUMEN FINAL:")
print(f"   - Nodos totales preparados: {len(nodes_final)}")
print(f"   - Nodos originales: {sum(1 for node in nodes_final.values() if node['es_original'])}")
print(f"   - Nodos intermedios: {sum(1 for node in nodes_final.values() if node['es_intermedio'])}")

# CREACIÓN DE DICCIONARIO DE ELEMENTOS CON PROPIEDADES INTERPOLADAS
# ============================================================


# CREAR EL DICCIONARIO DE ELEMENTOS
# ============================================================

print("\\n🏗️ CREANDO DICCIONARIO DE ELEMENTOS...")
elementos_dict = crear_diccionario_elementos(tramos_propiedades, nodes_final, nodes_key)

# MOSTRAR RESUMEN
# ============================================================

print(f"\\n✅ DICCIONARIO DE ELEMENTOS CREADO EXITOSAMENTE")
print(f"📊 RESUMEN DE ELEMENTOS POR TRAMO:")

elementos_por_tramo = {}
for elem_id, elem_info in elementos_dict.items():
    tramo = elem_info['tramo']
    if tramo not in elementos_por_tramo:
        elementos_por_tramo[tramo] = 0
    elementos_por_tramo[tramo] += 1

for tramo, cantidad in elementos_por_tramo.items():
    print(f"   - {tramo}: {cantidad} elementos")

print(f"\\n🔧 TOTAL DE ELEMENTOS: {len(elementos_dict)}")

# MOSTRAR EJEMPLO DE UN ELEMENTO
# ============================================================

if elementos_dict:
    primer_elem = elementos_dict[1]
    print(f"\\n📋 EJEMPLO DE ELEMENTO (ID: {primer_elem['id']}):")
    print(f"   Tramo: {primer_elem['tramo']}")
    print(f"   Nodos: {primer_elem['nombres_nodos']} (Tags: {primer_elem['nodos']})")
    print(f"   Longitud: {primer_elem['longitud']:.3f} m")
    print(f"   Posición relativa en tramo: {primer_elem['posicion_relativa']:.3f}")
    print(f"   Propiedades efectivas:")
    print(f"     - A = {primer_elem['propiedades']['A_eff']:.6f} m²")
    print(f"     - I = {primer_elem['propiedades']['I_eff']:.6f} m⁴")
    print(f"     - J = {primer_elem['propiedades']['J_eff']:.6f} m⁴")
    print(f"     - E = {primer_elem['propiedades']['E_eff']/1e9:.1f} GPa")
    print(f"     - G = {primer_elem['propiedades']['G_eff']/1e9:.1f} GPa")
    print(f"   Datos para Solver: {primer_elem['datos_Solver']}")


In [ ]:
# CELDA 204: VISUALIZACIÓN DE DATOS PRE CALCULO - NODOS, TRAMOS Y ELEMENTOS
# ============================================================
# ============================================================
# CONFIGURACIÓN
# ============================================================

ZOOM_FACTOR = 0.7
SAVE_IMGS = False

RESTRAINT_COLORS = {
    'FIXED': 'red', 'HINGED': 'blue', 'ROLLER_X': 'yellow', 
    'ROLLER_Y': 'yellow', 'ROLLER_Z': 'yellow', 'FREE': 'green'
}

tramos_unicos = list(tramos_propiedades.keys())
colores_base = generar_colores_distintos(len(tramos_unicos))
colores_ajustados = ajustar_colores_para_distincion(colores_base, umbral_minimo=0.4)
COLORES_TRAMOS = dict(zip(tramos_unicos, colores_ajustados))

print("🎨 COLORES POR TRAMO:")
for tramo, color in COLORES_TRAMOS.items():
    print(f"   {tramo}: RGB{color}")

# ============================================================
# FUNCIONES PRINCIPALES
# ============================================================

import os
import glob

def obtener_carpeta_plots():
    """Obtiene la carpeta para guardar plots"""
    try:
        notebook_files = glob.glob("*.ipynb")
        if notebook_files:
            notebook_name = os.path.splitext(notebook_files[0])[0]
            folder_name = f"{notebook_name}_plots"
            if not os.path.exists(folder_name):
                os.makedirs(folder_name)
            return folder_name
        else:
            if not os.path.exists(f"{tipo_estructura_lower}_notebook_plots"):
                os.makedirs(f"{tipo_estructura_lower}_notebook_plots")
            return f"{tipo_estructura_lower}_notebook_plots"
    except:
        if not os.path.exists("plots"):
            os.makedirs("plots")
        return "plots"

PLOTS_FOLDER = obtener_carpeta_plots() if SAVE_IMGS else ""

def calcular_rangos_ejes(nodes_final, zoom_factor=1.0):
    x_coords = [info['coordenadas']['x'] for info in nodes_final.values()]
    y_coords = [info['coordenadas']['y'] for info in nodes_final.values()]
    z_coords = [info['coordenadas']['z'] for info in nodes_final.values()]
    
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)
    z_min, z_max = min(z_coords), max(z_coords)
    
    max_range = max(x_max - x_min, y_max - y_min, z_max - z_min)
    zoomed_range = max_range * zoom_factor
    margin = max_range * 0.1
    
    return {
        'x_center': (x_max + x_min) / 2, 'y_center': (y_max + y_min) / 2, 'z_center': (z_max + z_min) / 2,
        'max_range': max_range, 'zoomed_range': zoomed_range, 'margin': margin
    }

def configurar_ejes_3d(ax, rangos, zoom_factor=1.0):
    effective_range = rangos['zoomed_range'] + rangos['margin']
    ax.set_xlim(rangos['x_center'] - effective_range/2, rangos['x_center'] + effective_range/2)
    ax.set_ylim(rangos['y_center'] - effective_range/2, rangos['y_center'] + effective_range/2)
    ax.set_zlim(rangos['z_center'] - effective_range/2, rangos['z_center'] + effective_range/2)
    ax.set_box_aspect([1, 1, 1])
    ax.invert_xaxis()

def dibujar_elementos_3d(ax, elementos_dict, nodes_final, color=None, linewidth=2):
    for elem_info in elementos_dict.values():
        tramo = elem_info['tramo']
        nodoA, nodoB = elem_info['nodos']
        coordsA, coordsB = nodes_final[nodoA]['coordenadas'], nodes_final[nodoB]['coordenadas']
        
        x_line = [coordsA['x'], coordsB['x']]
        y_line = [coordsA['y'], coordsB['y']]
        z_line = [coordsA['z'], coordsB['z']]
        
        elem_color = color if color else COLORES_TRAMOS[tramo]
        ax.plot(x_line, y_line, z_line, color=elem_color, linewidth=linewidth, alpha=0.8)

def dibujar_flechas_cargas(ax, cargas_hipotesis, nodes_final, rangos, longitud_fija=0.12):
    colores = {'x': 'red', 'y': 'green', 'z': 'blue'}
    
    for nombre_nodo, carga in cargas_hipotesis.items():
        if any(val != 0 for val in carga):
            nodo_tag = next((tag for tag, info in nodes_final.items() if info['nombre'] == nombre_nodo), None)
            if nodo_tag:
                coords = nodes_final[nodo_tag]['coordenadas']
                x, y, z = coords['x'], coords['y'], coords['z']
                
                for i, (componente, color) in enumerate(colores.items()):
                    fuerza = carga[i]
                    if fuerza != 0:
                        direccion = np.zeros(3)
                        direccion[i] = 1 if fuerza > 0 else -1
                        longitud = longitud_fija * rangos['max_range']
                        
                        ax.quiver(x, y, z, 
                                 direccion[0] * longitud, direccion[1] * longitud, direccion[2] * longitud,
                                 color=color, arrow_length_ratio=0.3, linewidth=1.5, alpha=0.9)
                        
                        fuerza_daN = abs(fuerza) / 10
                        offset = 0.15 * longitud
                        x_text = x + direccion[0] * (longitud + offset)
                        y_text = y + direccion[1] * (longitud + offset)
                        z_text = z + direccion[2] * (longitud + offset)
                        
                        ax.text(x_text, y_text, z_text, f"{fuerza_daN:.0f}", 
                               color=color, fontsize=8, fontweight='bold',
                               bbox=dict(boxstyle="round,pad=0.1", facecolor="white", alpha=0.9))

def obtener_nodos_con_cargas(cargas_hipotesis):
    return [nombre for nombre, carga in cargas_hipotesis.items() if any(val != 0 for val in carga)]

def guardar_figura(fig, nombre_archivo, dpi=150):
    if SAVE_IMGS:
        try:
            ruta = os.path.join(PLOTS_FOLDER, nombre_archivo)
            fig.savefig(ruta, dpi=dpi, bbox_inches='tight', facecolor='white', edgecolor='none')
        except Exception as e:
            print(f"Error al guardar {nombre_archivo}: {e}")

# ============================================================
# GRÁFICOS
# ============================================================

rangos = calcular_rangos_ejes(nodes_final, ZOOM_FACTOR)

# Gráfico 1: Nodos con restricciones
fig1 = plt.figure(figsize=(15, 12))
ax1 = fig1.add_subplot(111, projection='3d')

for tag, node_info in nodes_final.items():
    coords = node_info['coordenadas']
    color = RESTRAINT_COLORS[node_info['tipo_restriccion']]
    ax1.scatter(coords['x'], coords['y'], coords['z'], c=color, s=100, alpha=0.7)

for tag, node_info in nodes_final.items():
    if node_info['nombre'] in NODOS_ETIQUETAR:
        coords = node_info['coordenadas']
        ax1.text(coords['x'], coords['y'], coords['z'] + 0.5, node_info['nombre'], fontsize=8, fontweight='bold')

configurar_ejes_3d(ax1, rangos, ZOOM_FACTOR)
ax1.set_xlabel('X (m)'); ax1.set_ylabel('Y (m)'); ax1.set_zlabel('Z (m)')
ax1.set_title('Distribución de Nodos con Restricciones', fontsize=14, fontweight='bold')

legend_elements = [Patch(facecolor=color, label=tipo) for tipo, color in RESTRAINT_COLORS.items()]
ax1.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.98, 0.98))

plt.tight_layout()
plt.show()
guardar_figura(fig1, "01_nodos_con_restricciones.png")

# Gráfico 2: Elementos por tramo
fig2 = plt.figure(figsize=(15, 12))
ax2 = fig2.add_subplot(111, projection='3d')

dibujar_elementos_3d(ax2, elementos_dict, nodes_final, linewidth=2.5)

for tag, node_info in nodes_final.items():
    if node_info['nombre'] in NODOS_ETIQUETAR:
        coords = node_info['coordenadas']
        ax2.text(coords['x'], coords['y'], coords['z'] + 0.5, node_info['nombre'], fontsize=8, fontweight='bold')

configurar_ejes_3d(ax2, rangos, ZOOM_FACTOR)
ax2.set_xlabel('X (m)'); ax2.set_ylabel('Y (m)'); ax2.set_zlabel('Z (m)')
ax2.set_title('Elementos de Estructura por Tramo', fontsize=14, fontweight='bold')

legend_elements = [Patch(facecolor=COLORES_TRAMOS[tramo], label=tramo) for tramo in tramos_unicos]
ax2.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.98, 0.98))

plt.tight_layout()
plt.show()
guardar_figura(fig2, "02_elementos_por_tramo.png")


print(f"\n📊 RESUMEN:")
print(f"   - Nodos: {len(nodes_final)}")
print(f"   - Elementos: {len(elementos_dict)}")
print(f"   - Hipótesis: {len(cargas_key)}")
print(f"   - Tramos: {len(tramos_unicos)}")
print(f"   - Zoom: {ZOOM_FACTOR}")

if SAVE_IMGS:
    print(f"   - Imágenes guardadas en: {PLOTS_FOLDER}")



In [ ]:
### 007: ÁRBOLES DE CARGA 2D
# ============================================================
# CONFIGURACIÓN GLOBAL
# ============================================================

GRAFICO_3D = False  # True para 3D, False para 2D

# Configuración común
ZOOM_FACTOR = 0.5 if not GRAFICO_3D else 0.7
SAVE_IMGS = True
VER_NODOS = True
VER_NODOS_CARGADOS = True
DEFASAR_NODOS = True if not GRAFICO_3D else False
DESFASE = (-0.3, 0.2) if not GRAFICO_3D else (0, 0)

# Configuración específica para 2D
ESCALA_FLECHA = 1.8 if not GRAFICO_3D else 1.0
RELACION_CABEZA_FLECHA = 0.2 if not GRAFICO_3D else 0.3
GROSOR_FLECHA = 1.2 if not GRAFICO_3D else 1.0
ALPHA_FLECHA = 0.95 if not GRAFICO_3D else 0.8

# Definir nodos de apoyo
nodos_apoyo = ["BASE"]  # Nodo principal de apoyo

RESTRAINT_COLORS = {
    'FIXED': 'red', 'HINGED': 'blue', 'ROLLER_X': 'yellow', 
    'ROLLER_Y': 'yellow', 'ROLLER_Z': 'yellow', 'FREE': 'green'
}

print(f"🎯 MODO DE VISUALIZACIÓN: {'3D' if GRAFICO_3D else '2D'}")
print(f"📍 NODOS DE APOYO: {nodos_apoyo}")

# ============================================================
# OBTENER DATOS DE REACCIONES DESDE ESTRUCTURA_POO
# ============================================================

# Inicializar diccionario de reacciones
resultados_reacciones = {}

# Verificar si tenemos datos de reacciones en estructura_poo
if hasattr(estructura_poo, 'resultados_reacciones'):
    print("✅ Usando datos de reacciones de estructura_poo.resultados_reacciones")
    resultados_reacciones = estructura_poo.resultados_reacciones
elif hasattr(estructura_poo, 'df_reacciones'):
    print("✅ Usando datos de reacciones de estructura_poo.df_reacciones")
    # Convertir DataFrame a diccionario
    for hipotesis, datos in estructura_poo.df_reacciones.iterrows():
        resultados_reacciones[hipotesis] = {
            'Reaccion_Fx_daN': datos['Reaccion_Fx_daN'],
            'Reaccion_Fy_daN': datos['Reaccion_Fy_daN'], 
            'Reaccion_Fz_daN': datos['Reaccion_Fz_daN'],
            'Reaccion_Mx_daN_m': datos['Reaccion_Mx_daN_m'],
            'Reaccion_My_daN_m': datos['Reaccion_My_daN_m'],
            'Reaccion_Mz_daN_m': datos['Reaccion_Mz_daN_m'],
            'Tiro_resultante_daN': datos['Tiro_resultante_daN']
        }
else:
    print("⚠️  No se encontraron datos de reacciones. Usando valores por defecto.")
    # Crear datos por defecto para evitar errores
    for hipotesis_nombre in cargas_key.keys():
        resultados_reacciones[hipotesis_nombre] = {
            'Reaccion_Fx_daN': 0, 'Reaccion_Fy_daN': 0, 'Reaccion_Fz_daN': 0,
            'Reaccion_Mx_daN_m': 0, 'Reaccion_My_daN_m': 0, 'Reaccion_Mz_daN_m': 0,
            'Tiro_resultante_daN': 0
        }

print(f"📊 Datos de reacciones cargados para {len(resultados_reacciones)} hipótesis")

# ============================================================
# FUNCIONES PRINCIPALES UNIFICADAS
# ============================================================

import os
import glob
import numpy as np
from matplotlib.patches import Patch, FancyArrow

PLOTS_FOLDER = folder

def calcular_rangos_ejes(nodes_final, zoom_factor=1.0, grafico_3d=False):
    """Calcula rangos de ejes optimizados para 3D o 2D"""
    x_coords = [info['coordenadas']['x'] for info in nodes_final.values()]
    y_coords = [info['coordenadas']['y'] for info in nodes_final.values()]
    z_coords = [info['coordenadas']['z'] for info in nodes_final.values()]
    
    if grafico_3d:
        # Modo 3D
        return {
            'x_center': (max(x_coords) + min(x_coords)) / 2,
            'y_center': (max(y_coords) + min(y_coords)) / 2,
            'z_center': (max(z_coords) + min(z_coords)) / 2,
            'max_range': max(max(x_coords)-min(x_coords), max(y_coords)-min(y_coords), max(z_coords)-min(z_coords)),
            'zoomed_range': max(max(x_coords)-min(x_coords), max(y_coords)-min(y_coords), max(z_coords)-min(z_coords)) * zoom_factor,
            'margin': max(max(x_coords)-min(x_coords), max(y_coords)-min(y_coords), max(z_coords)-min(z_coords)) * 0.1
        }
    else:
        # Modo 2D (plano XZ) - Z siempre comienza en 0, termina en 1.2 * z_max
        z_max = max(z_coords)
        z_min = 0
        z_range = max(z_max * 1.05 - z_min, 10)
        
        x_min, x_max = min(x_coords), max(x_coords)
        x_range = x_max - x_min
        
        max_range = max(x_range, z_range) * zoom_factor
        margin = max_range * 0.1
        
        x_center = (x_max + x_min) / 2
        z_center = (z_max + z_min) / 2
        
        return {
            'x_min': x_center - max_range/2 - margin,
            'x_max': x_center + max_range/2 + margin,
            'z_min': z_min,
            'z_max': max(z_max * 1.2, max_range/2 + z_center),
            'x_center': x_center,
            'z_center': z_center,
            'max_range': max_range,
            'z_max_original': z_max
        }

def configurar_ejes(ax, rangos, grafico_3d=False):
    """Configura ejes según el modo 3D o 2D"""
    if grafico_3d:
        effective_range = rangos['zoomed_range'] + rangos['margin']
        ax.set_xlim(rangos['x_center'] - effective_range/2, rangos['x_center'] + effective_range/2)
        ax.set_ylim(rangos['y_center'] - effective_range/2, rangos['y_center'] + effective_range/2)
        ax.set_zlim(rangos['z_center'] - effective_range/2, rangos['z_center'] + effective_range/2)
        ax.set_box_aspect([1, 1, 1])
        ax.invert_xaxis()
        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_zlabel('Z (m)')
    else:
        ax.set_xlim(rangos['x_min'], rangos['x_max'])
        ax.set_ylim(rangos['z_min'], rangos['z_max'])
        ax.set_aspect('equal')
        ax.grid(True, alpha=0.3)
        ax.set_xlabel('X (m)')
        ax.set_ylabel('Z (m)')

def dibujar_elementos(ax, elementos_dict, nodes_final, color='black', linewidth=2, grafico_3d=False):
    """Dibuja elementos según el modo 3D o 2D - TODOS EN NEGRO"""
    for elem_info in elementos_dict.values():
        nodoA, nodoB = elem_info['nodos']
        coordsA, coordsB = nodes_final[nodoA]['coordenadas'], nodes_final[nodoB]['coordenadas']
        
        elem_color = color
        
        if grafico_3d:
            ax.plot([coordsA['x'], coordsB['x']], 
                    [coordsA['y'], coordsB['y']], 
                    [coordsA['z'], coordsB['z']], 
                    color=elem_color, linewidth=linewidth, alpha=0.8)
        else:
            ax.plot([coordsA['x'], coordsB['x']], 
                    [coordsA['z'], coordsB['z']], 
                    color=elem_color, linewidth=linewidth, alpha=0.8, solid_capstyle='round')

def dibujar_flechas_cargas(ax, cargas_hipotesis, nodes_final, rangos, grafico_3d=False, 
                          escala=1.0, relacion_cabeza=0.25, longitud_base=0.08):
    """Dibuja flechas de carga según el modo 3D o 2D"""
    colores = {'x': 'red', 'y': 'green', 'z': 'blue'}
    
    for nombre_nodo, carga in cargas_hipotesis.items():
        if not any(val != 0 for val in carga):
            continue
            
        nodo_tag = next((tag for tag, info in nodes_final.items() if info['nombre'] == nombre_nodo), None)
        if not nodo_tag:
            continue
            
        coords = nodes_final[nodo_tag]['coordenadas']
        
        if grafico_3d:
            # Modo 3D
            x, y, z = coords['x'], coords['y'], coords['z']
            longitud = longitud_base * rangos['max_range']
            
            for i, (componente, color) in enumerate(colores.items()):
                fuerza = carga[i]
                if fuerza != 0:
                    direccion = np.zeros(3)
                    direccion[i] = 1 if fuerza > 0 else -1
                    
                    # Calcular posición media del vector
                    x_medio = x + direccion[0] * longitud * 0.5
                    y_medio = y + direccion[1] * longitud * 0.5
                    z_medio = z + direccion[2] * longitud * 0.5
                    
                    # Dibujar flecha
                    ax.quiver(x, y, z, 
                             direccion[0] * longitud, direccion[1] * longitud, direccion[2] * longitud,
                             color=color, arrow_length_ratio=0.3, linewidth=1.5, alpha=0.9)
                    
                    # Colocar etiqueta en medio del vector
                    fuerza_daN = abs(fuerza) / 10
                    ax.text(x_medio, y_medio, z_medio, f"{fuerza_daN:.1f}", 
                           color=color, fontsize=9, fontweight='bold', zorder=1000,
                           bbox=dict(boxstyle="round,pad=0.2", facecolor="white", alpha=0.95, edgecolor=color),
                           ha='center', va='center')
        else:
            # Modo 2D
            x, z = coords['x'], coords['z']
            longitud = longitud_base * rangos['max_range'] * escala
            
            for i, (componente, color) in enumerate(colores.items()):
                fuerza = carga[i]
                if fuerza != 0:
                    fuerza_daN = abs(fuerza) / 10
                    direccion = 1 if fuerza > 0 else -1
                    
                    if componente == 'x':  # Fuerza en X
                        dx, dz = direccion * longitud, 0
                        ax.arrow(x, z, dx, dz, 
                               head_width=longitud*0.12,
                               head_length=longitud*relacion_cabeza,
                               fc=color, ec=color, linewidth=GROSOR_FLECHA, alpha=ALPHA_FLECHA,
                               length_includes_head=True, zorder=500, overhang=0.8)
                        ax.text(x + dx/2, z + longitud*0.1, f"{fuerza_daN:.1f}", 
                               color='darkred', fontsize=9, fontweight='bold', zorder=1000,
                               bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.95, 
                                       edgecolor=color, linewidth=1.2),
                               ha='center', va='center')
                    
                    elif componente == 'z':  # Fuerza en Z
                        dx, dz = 0, direccion * longitud
                        ax.arrow(x, z, dx, dz, 
                               head_width=longitud*0.12,
                               head_length=longitud*relacion_cabeza,
                               fc=color, ec=color, linewidth=GROSOR_FLECHA, alpha=ALPHA_FLECHA,
                               length_includes_head=True, zorder=500, overhang=0.8)
                        ax.text(x + longitud*0.1, z + dz/2, f"{fuerza_daN:.1f}", 
                               color='darkblue', fontsize=9, fontweight='bold', zorder=1000,
                               bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.95, 
                                       edgecolor=color, linewidth=1.2),
                               ha='center', va='center')
                    
                    elif componente == 'y':  # Fuerza en Y - representada a 45°
                        dx = direccion * longitud * 0.7
                        dz = direccion * longitud * 0.7
                        ax.arrow(x, z, dx, dz, 
                               head_width=longitud*0.12,
                               head_length=longitud*relacion_cabeza,
                               fc=color, ec=color, linewidth=GROSOR_FLECHA, alpha=ALPHA_FLECHA, 
                               linestyle='--', length_includes_head=True, zorder=500, overhang=0.8)
                        ax.text(x + dx/2, z + dz/2 + longitud*0.1, f"{fuerza_daN:.1f}", 
                               color='darkgreen', fontsize=9, fontweight='bold', zorder=1000,
                               bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.95, 
                                       edgecolor=color, linewidth=1.2),
                               ha='center', va='center')

def obtener_nodos_con_cargas(cargas_hipotesis):
    """Obtiene nodos con cargas aplicadas"""
    nodos_cargados = [nombre for nombre, carga in cargas_hipotesis.items() if any(val != 0 for val in carga)]
    return nodos_cargados

def dibujar_panel_reacciones(ax, datos_reacciones, rangos, nodo_base_coords, grafico_3d=False):
    """Dibuja panel informativo con reacciones"""
    if grafico_3d:
        # Modo 3D: debajo del nodo base
        x_pos = nodo_base_coords['x']
        y_pos = nodo_base_coords['y'] 
        z_pos = nodo_base_coords['z'] - rangos['max_range'] * 0.15
        
        texto = f"REACCIONES BASE:\n"
        texto += f"Fx: {datos_reacciones['Reaccion_Fx_daN']:.1f} daN\n"
        texto += f"Fy: {datos_reacciones['Reaccion_Fy_daN']:.1f} daN\n"
        texto += f"Fz: {datos_reacciones['Reaccion_Fz_daN']:.1f} daN\n"
        texto += f"Mx: {datos_reacciones['Reaccion_Mx_daN_m']:.1f} daN·m\n"
        texto += f"My: {datos_reacciones['Reaccion_My_daN_m']:.1f} daN·m\n"
        texto += f"Mz: {datos_reacciones['Reaccion_Mz_daN_m']:.1f} daN·m\n"
        texto += f"Fcima: {datos_reacciones['Tiro_resultante_daN']:.1f} daN"
        
        ax.text(x_pos, y_pos, z_pos, texto, 
                fontsize=8, fontweight='bold', zorder=1000, color='white',
                bbox=dict(boxstyle="round,pad=0.5", facecolor="black", alpha=0.8),
                ha='left', va='top')
    else:
        # Modo 2D: esquina inferior izquierda
        x_pos = rangos['x_min'] + rangos['max_range'] * 0.02
        z_pos = rangos['z_min'] + rangos['max_range'] * 0.02
        
        texto = f"REACCIONES BASE:\n"
        texto += f"Fx: {datos_reacciones['Reaccion_Fx_daN']:.1f} daN\n"
        texto += f"Fy: {datos_reacciones['Reaccion_Fy_daN']:.1f} daN\n"
        texto += f"Fz: {datos_reacciones['Reaccion_Fz_daN']:.1f} daN\n"
        texto += f"Mx: {datos_reacciones['Reaccion_Mx_daN_m']:.1f} daN·m\n"
        texto += f"My: {datos_reacciones['Reaccion_My_daN_m']:.1f} daN·m\n"
        texto += f"Mz: {datos_reacciones['Reaccion_Mz_daN_m']:.1f} daN·m\n"
        texto += f"Fcima: {datos_reacciones['Tiro_resultante_daN']:.1f} daN"
        
        ax.text(x_pos, z_pos, texto, 
                fontsize=8, fontweight='bold', zorder=1000, color='black',
                bbox=dict(boxstyle="round,pad=0.5", facecolor="white", alpha=0.9, edgecolor='black'),
                ha='left', va='bottom')

def formatear_lista_nodos(nodos, nodos_por_linea=3):
    """Formatea lista de nodos con saltos de línea"""
    if not nodos:
        return ""
    
    lineas = []
    for i in range(0, len(nodos), nodos_por_linea):
        lineas.append(", ".join(nodos[i:i+nodos_por_linea]))
    return "\n".join(lineas)

def guardar_figura(fig, nombre_archivo, dpi=150):
    """Guarda figura optimizada"""
    if SAVE_IMGS:
        try:
            ruta = os.path.join(PLOTS_FOLDER, nombre_archivo)
            fig.savefig(ruta, dpi=dpi, bbox_inches='tight', facecolor='white', edgecolor='none')
            print(f"💾 Imagen guardada: {nombre_archivo}")
        except Exception as e:
            print(f"❌ Error al guardar {nombre_archivo}: {e}")

# ============================================================
# GRÁFICOS PRINCIPALES UNIFICADOS
# ============================================================

rangos = calcular_rangos_ejes(nodes_final, ZOOM_FACTOR, GRAFICO_3D)

print(f"📐 RANGOS CALCULADOS:")
print(f"   - X: [{rangos['x_min']:.1f}, {rangos['x_max']:.1f}]")
print(f"   - Z: [{rangos['z_min']:.1f}, {rangos['z_max']:.1f}]")
print(f"   - Z máximo original: {rangos['z_max_original']:.1f}")
print(f"   - Z máximo ajustado: {rangos['z_max']:.1f} (1.2×Z_max = {rangos['z_max_original'] * 1.05:.1f})")

# Verificar que hay datos de cargas
if not cargas_key:
    print("❌ ERROR: No hay datos de cargas definidos")
else:
    print(f"📊 Procesando {len(cargas_key)} hipótesis de carga...")

# Gráficos por hipótesis
for idx, (hipotesis_nombre, cargas_hipotesis) in enumerate(cargas_key.items(), 1):
    print(f"\n📋 Procesando hipótesis {idx}: {hipotesis_nombre}")
    
    # Verificar si hay fuerzas en esta hipótesis
    nodos_con_cargas = obtener_nodos_con_cargas(cargas_hipotesis)
    if not nodos_con_cargas:
        print(f"   ⚠️  ADVERTENCIA: No hay fuerzas aplicadas en la hipótesis '{hipotesis_nombre}'")
        continue
    
    # OBTENER DATOS DE REACCIONES
    if hipotesis_nombre in resultados_reacciones:
        datos_reacciones = resultados_reacciones[hipotesis_nombre]
        print(f"   ✅ Datos de reacciones encontrados")
    else:
        print(f"   ❌ ERROR: No hay datos de reacciones para '{hipotesis_nombre}'")
        datos_reacciones = {
            'Reaccion_Fx_daN': 0, 'Reaccion_Fy_daN': 0, 'Reaccion_Fz_daN': 0,
            'Reaccion_Mx_daN_m': 0, 'Reaccion_My_daN_m': 0, 'Reaccion_Mz_daN_m': 0,
            'Tiro_resultante_daN': 0
        }
    
    # Obtener coordenadas del nodo base para el panel de reacciones
    nodo_base_nombre = nodos_apoyo[0] if nodos_apoyo else list(nodes_key.keys())[0]
    nodo_base_coords = None
    for tag, info in nodes_final.items():
        if info['nombre'] == nodo_base_nombre:
            nodo_base_coords = info['coordenadas']
            break
    
    if nodo_base_coords is None:
        nodo_base_coords = {'x': rangos['x_center'], 'y': rangos['y_center'], 'z': rangos['z_center']} if GRAFICO_3D else {'x': rangos['x_center'], 'z': rangos['z_center']}
    
    # CREAR GRÁFICO
    if GRAFICO_3D:
        fig = plt.figure(figsize=(15, 12))
        ax = fig.add_subplot(111, projection='3d')
    else:
        fig = plt.figure(figsize=(12, 10))
        ax = fig.add_subplot(111)
    
    # Dibujar elementos - TODOS EN NEGRO
    dibujar_elementos(ax, elementos_dict, nodes_final, color='black', linewidth=3.5, grafico_3d=GRAFICO_3D)
    
    # Dibujar cargas
    if GRAFICO_3D:
        dibujar_flechas_cargas(ax, cargas_hipotesis, nodes_final, rangos, grafico_3d=GRAFICO_3D, longitud_base=0.1)
    else:
        dibujar_flechas_cargas(ax, cargas_hipotesis, nodes_final, rangos, grafico_3d=GRAFICO_3D, 
                              escala=ESCALA_FLECHA, relacion_cabeza=RELACION_CABEZA_FLECHA, longitud_base=0.08)
    
    # Añadir panel de reacciones
    dibujar_panel_reacciones(ax, datos_reacciones, rangos, nodo_base_coords, grafico_3d=GRAFICO_3D)
    
    # Mostrar etiquetas de nodos cargados
    if VER_NODOS_CARGADOS:
        for tag, node_info in nodes_final.items():
            if node_info['nombre'] in nodos_con_cargas:
                coords = node_info['coordenadas']
                if GRAFICO_3D:
                    ax.text(coords['x'], coords['y'], coords['z'], node_info['nombre'], 
                           fontsize=8, fontweight='bold', ha='center', va='center',
                           bbox=dict(boxstyle="round,pad=0.2", facecolor="yellow", zorder=1010, alpha=0.7))
                else:
                    x_text = coords['x'] + (DESFASE[0] if DEFASAR_NODOS else 0)
                    z_text = coords['z'] + (DESFASE[1] if DEFASAR_NODOS else 0)
                    ax.text(x_text, z_text, node_info['nombre'], 
                           fontsize=8, fontweight='bold', ha='center', va='center',
                           bbox=dict(boxstyle="round,pad=0.2", facecolor="yellow", alpha=0.7, edgecolor='black'))
    
    # Configurar ejes
    configurar_ejes(ax, rangos, grafico_3d=GRAFICO_3D)
    
    # Título y leyenda
    plano_text = " (Plano XZ)" if not GRAFICO_3D else ""
    ax.set_title(f'Distribución de Cargas - {hipotesis_nombre}{plano_text}', fontsize=14, fontweight='bold')
    
    # LEYENDA COMPLETA
    if GRAFICO_3D:
        legend_elements = [Patch(facecolor='red', label='Fuerza X'), 
                          Patch(facecolor='green', label='Fuerza Y'), 
                          Patch(facecolor='blue', label='Fuerza Z')]
        ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.98, 0.98))
    else:
        legend_elements = [
            Patch(facecolor='red', label='Fuerza X Transversal'),
            Patch(facecolor='blue', label='Fuerza Z Vertical'), 
            Patch(facecolor='green', label='Fuerza Y Longitudinal')
        ]
        ax.legend(handles=legend_elements, loc='upper right')
    
    # Información adicional
    total_x = sum(carga[0] for carga in cargas_hipotesis.values() if any(carga)) / 10
    total_y = sum(carga[1] for carga in cargas_hipotesis.values() if any(carga)) / 10
    total_z = sum(carga[2] for carga in cargas_hipotesis.values() if any(carga)) / 10
    
    if GRAFICO_3D:
        ax.text2D(0.02, 0.98, f"Cargas aplicadas: {len(nodos_con_cargas)} nodos", transform=ax.transAxes, fontsize=10,
                  bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
        ax.text2D(0.02, 0.93, f"Fuerza total X: {total_x:.1f} daN", transform=ax.transAxes, fontsize=10,
                  bbox=dict(boxstyle="round,pad=0.3", facecolor="lightcoral", alpha=0.8))
        ax.text2D(0.02, 0.88, f"Fuerza total Y: {total_y:.1f} daN", transform=ax.transAxes, fontsize=10,
                  bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.8))
        ax.text2D(0.02, 0.83, f"Fuerza total Z: {total_z:.1f} daN", transform=ax.transAxes, fontsize=10,
                  bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8))
        
        if VER_NODOS_CARGADOS and nodos_con_cargas:
            nodos_texto = formatear_lista_nodos(nodos_con_cargas)
            ax.text2D(0.02, 0.70, f"Nodos cargados:\n{nodos_texto}", transform=ax.transAxes, fontsize=8,
                      bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow", alpha=0.8))
    else:
        info_text = f"Nodos cargados: {len(nodos_con_cargas)}\n"
        info_text += f"Fuerza total X: {total_x:.1f} daN\n"
        info_text += f"Fuerza total Y: {total_y:.1f} daN\n"
        info_text += f"Fuerza total Z: {total_z:.1f} daN"
        
        ax.text(0.02, 0.98, info_text, transform=ax.transAxes, fontsize=10,
                bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.9),
                ha='left', va='top')
        
        if VER_NODOS_CARGADOS and nodos_con_cargas:
            nodos_texto = formatear_lista_nodos(nodos_con_cargas)
            ax.text(0.02, 0.75, f"Nodos cargados:\n{nodos_texto}", transform=ax.transAxes, fontsize=8,
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow", alpha=0.8),
                    ha='left', va='top')
    
    plt.tight_layout()
    plt.show()
    
    # Guardar imagen
    modo_text = "3d" if GRAFICO_3D else "2d"
    nombre_archivo = f"03_cargas_{idx:02d}_{hipotesis_nombre.lower().replace(' ', '_').replace('ó','o').replace('/','_')}_{modo_text}.png"
    guardar_figura(fig, nombre_archivo)

# ============================================================
# RESUMEN FINAL
# ============================================================

print(f"\n📊 RESUMEN FINAL:")
print(f"   - Modo: {'3D' if GRAFICO_3D else '2D'}")
print(f"   - Nodos: {len(nodes_final)}")
print(f"   - Elementos: {len(elementos_dict)}")
print(f"   - Hipótesis procesadas: {len([h for h, c in cargas_key.items() if any(any(val != 0 for val in carga) for carga in [c])])}")

if not GRAFICO_3D:
    print(f"🎯 CONFIGURACIÓN 2D MEJORADA:")
    print(f"   - Zoom: {ZOOM_FACTOR}")
    print(f"   - ESCALA_FLECHA: {ESCALA_FLECHA}")
    print(f"   - RELACION_CABEZA_FLECHA: {RELACION_CABEZA_FLECHA}")
    print(f"   - GROSOR_FLECHA: {GROSOR_FLECHA}")
    print(f"   - ALPHA_FLECHA: {ALPHA_FLECHA}")
    print(f"   - DEFASAR_NODOS: {DEFASAR_NODOS}")
    print(f"   - DESFASE: {DESFASE} metros")
    print(f"   - Plano: XZ")
    print(f"   - Eje Z: comienza en 0, mínimo {rangos['z_min']:.1f}m")
    print(f"   - Eje Z: termina en {rangos['z_max']:.1f}m (1.2×HG1)")
    print(f"   - Elementos: TODOS EN NEGRO")
    print(f"   - Leyenda: Incluye Fuerza X, Y, Z con colores")
    print(f"   - Panel reacciones: Incluye Mx, My, Mz, F resultante")
else:
    print(f"🎯 CONFIGURACIÓN 3D:")
    print(f"   - Zoom: {ZOOM_FACTOR}")
    print(f"   - VER_NODOS: {VER_NODOS}")
    print(f"   - VER_NODOS_CARGADOS: {VER_NODOS_CARGADOS}")

print(f"📍 VISUALIZACIÓN:")
print(f"   - VER_NODOS: {VER_NODOS}")
print(f"   - VER_NODOS_CARGADOS: {VER_NODOS_CARGADOS}")

if SAVE_IMGS:
    print(f"💾 Imágenes guardadas en: {PLOTS_FOLDER}")
else:
    print(f"💾 Guardado de imágenes: DESACTIVADO")